En este proceso se explica la forma en que pueden descargarse los datos disponibles en la [API](https://bchapi-am.developer.azure-api.net/) del Banco Central de Honduras (BCH), utilizando Julia.

# Acceso a los Datos

Una vez que se crea el registro de usuario (usar el botón "Suscribirse" en el [sitio web](https://bchapi-am.developer.azure-api.net/), puede verse el procedimiento de registro e inicio de sesión en [YouTube](https://www.youtube.com/watch?v=8ZBllMSsKw4)), se necesita obtener una clave (ver explicación en [YouTube](https://www.youtube.com/watch?v=mV90s74OCfc)), ejecutando una consulta al catálogo de indicadores. Favor sustituir en el código la clave actual por su clave asignada.

Para conocer los datos disponibles, se debe ejecutar una consulta al catálogo de indicadores, que contiene información sobre las series disponibles:

* Nombre (código)
* Descripción
* Periodicidad
* Grupo
* Correlativo del Grupo


In [1]:
#import Pkg; Pkg.add("IMFData")
using Azure,Chain,CSV,DataFrames,DataFramesMeta,Dates,HTTP,JSON,JSON3

wd = @__DIR__
clave = "ff34cff7b0024ea39eb565fccb9f03b6" #Favor ingresar la clave proporcionada.

"ff34cff7b0024ea39eb565fccb9f03b6"

In [2]:
function count_groups(x)
  # Función para contar separaciones de grupo tomando caracter "-".
  c = 5
  @inbounds for i in eachindex(x)
      if ismissing(x[i])
          c -= 1
      end
  end
  return c
end

function count_description(x)
  # Función para contar separaciones de indicador tomando caracter "-".
  c = 8
  @inbounds for i in eachindex(x)
      if ismissing(x[i])
          c -= 1
      end
  end
  return c
end

count_description (generic function with 1 method)

# Formatos de consulta

## 1. Consulta de catálogo de indicadores

Consulta el catálogo de indicadores publicados mediante la Web-API, asimismo, se obtiene el detalle de la información del indicador: Id, nombre, descripción, grupo, correlativo de grupo y periodicidad. Ningún parámetro es obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores[?formato][&nombre][&descripcion][&periodicidad][&ordenamiento][&grupo][&correlativogrupo][&omitir][&reciente]`

Este es el listado de todos los indicadores disponibles:


In [3]:
function get_indicators(key)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/?formato=Json"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:CorrelativoGrupo,:Grupo,:Periodicidad]]
end

function get_indicators_filter(key,url)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:CorrelativoGrupo,:Grupo,:Periodicidad]]
end

get_indicators_filter (generic function with 1 method)

In [4]:
key = clave
df = get_indicators(key)
df = hcat(
  df,
  DataFrames.select(
    df, 
    :Descripcion =>
      ByRow(x -> get.(Ref(split(x, '-')), 1:8, missing)) =>
      [:Niv_Descr_1,:Niv_Descr_2,:Niv_Descr_3,:Niv_Descr_4,:Niv_Descr_5,:Niv_Descr_6,:Niv_Descr_7,:Niv_Descr_8]))
DataFrames.transform!(
  df, 
  AsTable(Between(:Niv_Descr_1,:Niv_Descr_8)) => ByRow(count_description) => :Niv_Descr)
df = hcat(
  df,
  DataFrames.select(
    df, 
    :Grupo =>
      ByRow(x -> get.(Ref(split(x, '-')), 1:5, missing)) =>
      [:Niv_Gr_1,:Niv_Gr_2,:Niv_Gr_3,:Niv_Gr_4,:Niv_Gr_5]))
DataFrames.transform!(
  df, 
  AsTable(Between(:Niv_Gr_1,:Niv_Gr_5)) => ByRow(count_groups) => :Niv_Gr)
CSV.write(
    wd .* "/api/variables_julia.csv",
    delim = ";",
  df);

Consultar solamente los indicadores filtrados por nombre (puede usarse partes del nombre), por ejemplo, "ESR":


In [5]:
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/?formato=Json&nombre=ESR"
get_indicators_filter(key,url)

Row,Id,Nombre,Descripcion,CorrelativoGrupo,Grupo,Periodicidad
,Int64,String,String,String,String,String
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,15,ESR-PIBT-PROD-DES-CONST,Trimestral
2,2,ESR-PIBT-PROD-DES-CORR-12,Serie Desestacionalizada-VALORES CORRIENTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,12,ESR-PIBT-PROD-DES-CORR,Trimestral
3,3,ESR-PIBT-PROD-DES-CORR-13,Serie Desestacionalizada-VALORES CORRIENTES-16-Servicios Comunitarios Sociales y Personales,13,ESR-PIBT-PROD-DES-CORR,Trimestral
4,4,ESR-PIBT-PROD-DES-CORR-14,Serie Desestacionalizada-VALORES CORRIENTES-20-Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI),14,ESR-PIBT-PROD-DES-CORR,Trimestral
5,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,15,ESR-PIBT-PROD-DES-CORR,Trimestral
6,6,ESR-PIBT-PROD-DES-CORR-16,Serie Desestacionalizada-VALORES CORRIENTES-22-Más Impuestos netos de subvenciones sobre la producción y las importaciones,16,ESR-PIBT-PROD-DES-CORR,Trimestral
7,7,ESR-PIBT-PROD-DES-CORR-17,Serie Desestacionalizada-VALORES CORRIENTES-23-Producto Interno Bruto a precios de mercado,17,ESR-PIBT-PROD-DES-CORR,Trimestral
8,8,ESR-PIBT-GAST-OG-CONST-1,Serie Original-VALORES CONSTANTES-1-Gastos de Consumo Final,1,ESR-PIBT-GAST-OG-CONST,Trimestral
9,9,ESR-PIBT-GAST-OG-CONST-2,Serie Original-VALORES CONSTANTES-2-Sector Privado,2,ESR-PIBT-GAST-OG-CONST,Trimestral


Esto aplica para el resto de los campos; un ejemplo de consulta con varios filtros:


In [6]:
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/?formato=Json&nombre=PIBA&correlativogrupo=10&Descripcion=Constantes"
get_indicators_filter(key,url)

Row,Id,Nombre,Descripcion,CorrelativoGrupo,Grupo,Periodicidad
,Int64,String,String,String,String,String
1,101,ESR-PIBA-01-10,"ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Agricultura, Ganadería, Caza, Silvicultura y Pesca",10,ESR-PIBA-01,Anual


Nota: tomar en cuenta para todas las consultas por filtro que si la ruta no existe, el código genera un error.

## 2. Consulta cifras por Id de indicador

Consulta las cifras para un indicador en específico obtenido por el Id del indicador. El parámetro "Id" es un campo obligatorio el resto de criterios quedan a consideración del usuario. Si no se especifican parámetros, se obtendrán todos los datos disponibles para el indicador requerido, lo que podría demorar la obtención de la consulta, en ese sentido lo que se sugiere afinar la búsqueda a los criterios requeridos.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras[?formato][&fechaInicio][&fechaFinal][&valorMinimo][&valorMaximo][&ordenamiento][&omitir][&reciente]`


In [7]:
function get_data_indicator(key,id)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" *  string(id) * "/cifras"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df = df[!,[:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:Fecha],rev=true)
  return df
end

function get_data_indicator_filter(key,url,id)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df = df[!,[:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:Fecha],rev=true)
  return df
end

function get_data_indicators(key,ids)
  ids = collect(ids)
  df = DataFrames.DataFrame()
  for i in ids
    df = vcat(df,get_data_indicator(key,i))
  end
  return df
end

get_data_indicators (generic function with 1 method)

Consultar solo un indicador:


In [8]:
id = 5
get_data_indicator(key,id)

Row,IndicadorId,Nombre,Descripcion,Fecha,Valor
,Int64,String,String,Date,Float64
1,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2023-12-01,1.97719e5
2,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2023-09-01,1.91436e5
3,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2023-06-01,1.89149e5
4,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2023-03-01,1.83722e5
5,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2022-12-01,1.82085e5
6,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2022-09-01,1.77787e5
7,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2022-06-01,1.7055e5
8,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2022-03-01,1.6403e5
9,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,2021-12-01,1.60639e5


Consultar varios indicadores, usando rangos:


In [9]:
ids = 1:10
get_data_indicators(key,ids)

Row,IndicadorId,Nombre,Descripcion,Fecha,Valor
,Int64,String,String,Date,Float64
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-12-01,56154.8
2,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-09-01,56164.6
3,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-06-01,55014.6
4,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-03-01,54039.6
5,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-12-01,54209.0
6,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-09-01,53993.7
7,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-06-01,53215.6
8,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-03-01,52517.1
9,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2021-12-01,52540.2


Consultar varios indicadores, usando IDs no contínuas:


In [10]:
ids = 1,7,25,36
get_data_indicators(key,ids)

Row,IndicadorId,Nombre,Descripcion,Fecha,Valor
,Int64,String,String,Date,Float64
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-12-01,56154.8
2,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-09-01,56164.6
3,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-06-01,55014.6
4,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-03-01,54039.6
5,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-12-01,54209.0
6,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-09-01,53993.7
7,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-06-01,53215.6
8,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2022-03-01,52517.1
9,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2021-12-01,52540.2


Consultar solo un indicador, usando filtros:


In [11]:
id = 36
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "/cifras?FechaInicio=2023-03-01&FechaFinal=2023-09-01"
get_data_indicator_filter(key,url,id)

Row,IndicadorId,Nombre,Descripcion,Fecha,Valor
,Int64,String,String,Date,Float64
1,36,ESR-PIBT-GAST-DES-CONST-3,Serie Desestacionalizada-VALORES CONSTANTES-3-Sector Público,2023-09-01,8098.2
2,36,ESR-PIBT-GAST-DES-CONST-3,Serie Desestacionalizada-VALORES CONSTANTES-3-Sector Público,2023-06-01,8090.8
3,36,ESR-PIBT-GAST-DES-CONST-3,Serie Desestacionalizada-VALORES CONSTANTES-3-Sector Público,2023-03-01,8219.4


## 3. Consulta Indicadores por grupo

Consulta un conjunto de indicadores publicados mediante la Web-API por grupo de indicadores. El parámetro "grupo" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/{grupo}[?formato]`


In [12]:
function get_groups(key)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores?formato=Json"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  DataFrames.sort!(df,:CorrelativoGrupo)
  df = df[!,[:Id,:Descripcion,:Periodicidad,:Grupo,:Nombre,:CorrelativoGrupo]]
  return df
end

function get_groups_filter(key,url)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  DataFrames.sort!(df,:Id)
  return df
end

get_groups_filter (generic function with 1 method)

Consultar lista de todos los indicadores:


In [13]:
get_groups(key)

Row,Id,Descripcion,Periodicidad,Grupo,Nombre,CorrelativoGrupo
,Int64,String,String,String,String,String
1,97,Estadísticas Cambiarias - Tipo de Cambio de Referencia,Diario,EC-TCR,EC-TCR-01,01
2,98,Valores Ajustados,Trimestral,ESR-PIBT-TDC-Valores,ESR-PIBT-TDC-Valores-01,01
3,8,Serie Original-VALORES CONSTANTES-1-Gastos de Consumo Final,Trimestral,ESR-PIBT-GAST-OG-CONST,ESR-PIBT-GAST-OG-CONST-1,1
4,21,Serie Desestacionalizada-VALORES CORRIENTES-1-Agricultura Ganadería Caza Silvicultura y Pesca,Trimestral,ESR-PIBT-PROD-DES-CORR,ESR-PIBT-PROD-DES-CORR-1,1
5,29,Serie Desestacionalizada-VALORES CORRIENTES-1-Gastos de Consumo Final,Trimestral,ESR-PIBT-GAST-DES-CORR,ESR-PIBT-GAST-DES-CORR-1,1
6,37,Serie Original-VALORES CORRIENTES-1-Gastos de Consumo Final,Trimestral,ESR-PIBT-GAST-OG-CORR,ESR-PIBT-GAST-OG-CORR-1,1
7,40,Serie Desestacionalizada-VALORES CONSTANTES-1-Gastos de Consumo Final,Trimestral,ESR-PIBT-GAST-DES-CONST,ESR-PIBT-GAST-DES-CONST-1,1
8,54,Serie Original-VALORES CORRIENTES-1-Agricultura Ganadería Caza Silvicultura y Pesca,Trimestral,ESR-PIBT-PROD-OG-CORR,ESR-PIBT-PROD-OG-CORR-1,1
9,71,Serie Original-VALORES CONSTANTES-1-Agricultura Ganadería Caza Silvicultura y Pesca,Trimestral,ESR-PIBT-PROD-OG-CONST,ESR-PIBT-PROD-OG-CONST-1,1


Consultar componentes de grupo por filtro (en este caso debe especificarse el nombre completo del grupo, de acuerdo con la tabla anterior; caso contrario, se generan errores):


In [14]:
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/ESR-PIBT-PROD-OG-CONST?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,50,12,ESR-PIBT-PROD-OG-CONST-12,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,Trimestral
2,55,17,ESR-PIBT-PROD-OG-CONST-17,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-23-Producto Interno Bruto a precios de mercado,Trimestral
3,56,16,ESR-PIBT-PROD-OG-CONST-16,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-22-Más Impuestos netos de subvenciones sobre la producción y las importaciones,Trimestral
4,57,15,ESR-PIBT-PROD-OG-CONST-15,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,Trimestral
5,58,14,ESR-PIBT-PROD-OG-CONST-14,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-20-Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI),Trimestral
6,59,13,ESR-PIBT-PROD-OG-CONST-13,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-16-Servicios Comunitarios Sociales y Personales,Trimestral
7,60,11,ESR-PIBT-PROD-OG-CONST-11,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-13-Administración Pública y Defensa Planes de Seguridad Social de Afiliación Obligatoria,Trimestral
8,62,10,ESR-PIBT-PROD-OG-CONST-10,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-11-Propiedad de Vivienda Actividades Inmobiliarias y Empresariales,Trimestral
9,63,9,ESR-PIBT-PROD-OG-CONST-9,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-10-Intermediación Financiera,Trimestral


## 4. Consulta cifras por grupo de indicadores

Consultar las cifras de todos los indicadores por grupo de indicadores. El parámetro "grupo" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/{grupo}/cifras[?formato]`

Debe especificarse el nombre completo del (los) grupo(s), de acuerdo con la tabla anterior; caso contrario, se generan errores.


In [15]:
function get_data_group(key,group)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" *  group * "/cifras"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df[!,:Grupo] .= group
  df = df[!,[:Grupo,:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:IndicadorId,:Fecha],rev=[false,true])
  return df
end

function get_data_groups(key,groups)
  df = DataFrames.DataFrame()
  for i in 1:size(groups)[1]
    df = vcat(df,get_data_group(key,groups[i]))
  end
  return df
end

get_data_groups (generic function with 1 method)

Consultar cifras de un solo grupo:


In [16]:
group = "ESR-PIBT-PROD-OG-CONST"
get_data_group(key,group)

Row,Grupo,IndicadorId,Nombre,Descripcion,Fecha,Valor
,String,Int64,String,String,Date,Float64
1,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-12-01,4697.1
2,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-09-01,4532.6
3,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-06-01,4427.1
4,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-03-01,4219.0
5,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-12-01,4517.8
6,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-09-01,4423.5
7,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-06-01,4366.9
8,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-03-01,4184.4
9,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2021-12-01,4478.7


Consultar cifras de varios grupos:


In [17]:
groups = ["ESR-PIBT-PROD-OG-CONST","ESE-PII-01"]
get_data_groups(key,groups)

Row,Grupo,IndicadorId,Nombre,Descripcion,Fecha,Valor
,String,Int64,String,String,Date,Float64
1,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-12-01,4697.1
2,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-09-01,4532.6
3,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-06-01,4427.1
4,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2023-03-01,4219.0
5,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-12-01,4517.8
6,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-09-01,4423.5
7,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-06-01,4366.9
8,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2022-03-01,4184.4
9,ESR-PIBT-PROD-OG-CONST,50,ESR-PIBT-PROD-OG-CONST-12,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,2021-12-01,4478.7


## 5. Consulta por fecha de registro

Consulta el valor de un indicador para una fecha específica. Todos los parámetros son obligatorios.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras/{fecha}[?formato]`


In [18]:
function get_data_date(key,id,date)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "/cifras/" * date * "?formato=JSON"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  myFormat = Dates.DateFormat("yyy-mm-ddTHH:MM:SS")
  df.Fecha = @. Dates.Date(Dates.DateTime.(df.Fecha ,myFormat))
  df[!,:Grupo] .= group
  df = df[!,[:Grupo,:IndicadorId,:Nombre,:Descripcion,:Fecha,:Valor]]
  DataFrames.sort!(df,[:IndicadorId,:Fecha],rev=[false,true])
  return df
end

get_data_date (generic function with 1 method)

Consultar solamente un Id y una fecha:


In [19]:
id = 1
date = "2023-03-01"
get_data_date(key,id,date)

Row,Grupo,IndicadorId,Nombre,Descripcion,Fecha,Valor
,String,Int64,String,String,Date,Float64
1,ESR-PIBT-PROD-OG-CONST,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,2023-03-01,54039.6


## 6. Consulta de información por Id de indicador

Consulta el detalle de la información del indicador: Id, nombre, descripción, grupo, correlativo de grupo y periodicidad. El parámetro "Id" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras/{fecha}[?formato]`


In [20]:
function get_info_id(key,id)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "?formato=JSON"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:Periodicidad,:CorrelativoGrupo,:Grupo]]
  return df
end

function get_info_ids(key,ids)
  ids = collect(ids)
  df = DataFrames.DataFrame()
  for i in ids
    df = vcat(df,get_info_id(key,i))
  end
  return df
end

get_info_ids (generic function with 1 method)

Consultar solamente un Id:


In [21]:
id = 1
get_info_id(key,id)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo
,Int64,String,String,String,String,String
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,Trimestral,15,ESR-PIBT-PROD-DES-CONST


Consultar varios Ids, rango:


In [22]:
ids = 1:3
get_info_ids(key,ids)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo
,Int64,String,String,String,String,String
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,Trimestral,15,ESR-PIBT-PROD-DES-CONST
2,2,ESR-PIBT-PROD-DES-CORR-12,Serie Desestacionalizada-VALORES CORRIENTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,Trimestral,12,ESR-PIBT-PROD-DES-CORR
3,3,ESR-PIBT-PROD-DES-CORR-13,Serie Desestacionalizada-VALORES CORRIENTES-16-Servicios Comunitarios Sociales y Personales,Trimestral,13,ESR-PIBT-PROD-DES-CORR


Consultar varios Ids, números:


In [23]:
ids = 1,7,25
get_info_ids(key,ids)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo
,Int64,String,String,String,String,String
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,Trimestral,15,ESR-PIBT-PROD-DES-CONST
2,7,ESR-PIBT-PROD-DES-CORR-17,Serie Desestacionalizada-VALORES CORRIENTES-23-Producto Interno Bruto a precios de mercado,Trimestral,17,ESR-PIBT-PROD-DES-CORR
3,25,ESR-PIBT-GAST-DES-CORR-7,Serie Desestacionalizada-VALORES CORRIENTES-7-Producto Interno Bruto a precios de mercado,Trimestral,7,ESR-PIBT-GAST-DES-CORR


## 7. Conteo de cifras para cada indicador

Consulta los indicadores publicados mediante la Web-API devolviendo el conteo de registros disponibles para cada uno. Todos los filtros son opcionales.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/conteos[?formato][&nombre][&descripcion][&periodicidad][&ordenamiento][&conteoMinimo][&conteoMaximo][&ordenamientoConteo][&grupo][&correlativogrupo][&omitir][&reciente]`

Esta forma de consulta permite filtrar por valores incompletos de los elementos nombre, descripcion y grupo.


In [24]:
function get_count_gids(key,url)
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:Periodicidad,:CorrelativoGrupo,:Grupo,:Conteo]]
  return df
end

get_count_gids (generic function with 1 method)

Filtro por nombre del indicador:


In [25]:
filtro = "EC-"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&nombre=" * filtro
get_count_gids(key,url)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo,Conteo
,Int64,String,String,String,String,String,Int64
1,97,EC-TCR-01,Estadísticas Cambiarias - Tipo de Cambio de Referencia,Diario,01,EC-TCR,1317
2,619,EC-TCN-01-1,Tipo de Cambio Nominal - Compra,Diario,1,EC-TCN-01,6143
3,620,EC-TCN-01-2,Tipo de Cambio Nominal - Venta,Diario,2,EC-TCN-01,6143


Filtro por descripción del indicador:


In [26]:
filtro = "cambio"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&descripcion=" * filtro
get_count_gids(key,url)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo,Conteo
,Int64,String,String,String,String,String,Int64
1,97,EC-TCR-01,Estadísticas Cambiarias - Tipo de Cambio de Referencia,Diario,01,EC-TCR,1317
2,619,EC-TCN-01-1,Tipo de Cambio Nominal - Compra,Diario,1,EC-TCN-01,6143
3,620,EC-TCN-01-2,Tipo de Cambio Nominal - Venta,Diario,2,EC-TCN-01,6143


Filtro por periodicidad del indicador:

- Diario;
- Mensual;
- Trimestral; y
- Anual.


In [27]:
filtro = "Mensual"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&periodicidad=" * filtro
get_count_gids(key,url)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo,Conteo
,Int64,String,String,String,String,String,Int64
1,337,ESR-IMAE-01-4,"ESR-IMAE-01 - Agricultura, Ganaderia, Silvicultura y Pesca - Acumulada",Mensual,4,ESR-IMAE-01,556
2,338,ESR-IMAE-01-16,"ESR-IMAE-01 - Agricultura, Ganaderia, Silvicultura y Pesca - Interanual",Mensual,16,ESR-IMAE-01,556
3,339,ESR-IMAE-01-1,"ESR-IMAE-01 - Agricultura, Ganaderia, Silvicultura y Pesca - Índice",Mensual,1,ESR-IMAE-01,580
4,340,ESR-IMAE-01-5,ESR-IMAE-01 - Minas y Canteras - Acumulada,Mensual,5,ESR-IMAE-01,556
5,341,ESR-IMAE-01-17,ESR-IMAE-01 - Minas y Canteras - Interanual,Mensual,17,ESR-IMAE-01,556
6,342,ESR-IMAE-01-12,ESR-IMAE-01 - Minas y Canteras - Índice,Mensual,12,ESR-IMAE-01,580
7,343,ESR-IMAE-01-6,ESR-IMAE-01 - Industria Manufacturera - Acumulada,Mensual,6,ESR-IMAE-01,556
8,344,ESR-IMAE-01-18,ESR-IMAE-01 - Industria Manufacturera - Interanual,Mensual,18,ESR-IMAE-01,556
9,345,ESR-IMAE-01-23,ESR-IMAE-01 - Industria Manufacturera - Índice,Mensual,23,ESR-IMAE-01,580


Filtro por grupo del indicador:


In [28]:
filtro = "ODA"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&grupo=" * filtro
get_count_gids(key,url)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo,Conteo
,Int64,String,String,String,String,String,Int64
1,11426,ESR-ODA-01-1,DEMANDA GLOBAL---DEMANDA EXTERNA-EN MILLONES DE LEMPIRAS CONSTANTES,Anual,1,ESR-ODA-01,24
2,11427,ESR-ODA-01-2,DEMANDA GLOBAL---DEMANDA EXTERNA-EN MILLONES DE LEMPIRAS CORRIENTES,Anual,2,ESR-ODA-01,24
3,11428,ESR-ODA-01-3,DEMANDA GLOBAL---DEMANDA EXTERNA-VARIACIONES RELATIVAS CONSTANTE,Anual,3,ESR-ODA-01,23
4,11429,ESR-ODA-01-4,DEMANDA GLOBAL---DEMANDA EXTERNA-VARIACIONES RELATIVAS CORRIENTES,Anual,4,ESR-ODA-01,23
5,11430,ESR-ODA-01-5,DEMANDA GLOBAL---DEMANDA GLOBAL-EN MILLONES DE LEMPIRAS CONSTANTES,Anual,5,ESR-ODA-01,24
6,11431,ESR-ODA-01-6,DEMANDA GLOBAL---DEMANDA GLOBAL-EN MILLONES DE LEMPIRAS CORRIENTES,Anual,6,ESR-ODA-01,24
7,11432,ESR-ODA-01-7,DEMANDA GLOBAL---DEMANDA GLOBAL-VARIACIONES RELATIVAS CONSTANTE,Anual,7,ESR-ODA-01,23
8,11433,ESR-ODA-01-8,DEMANDA GLOBAL---DEMANDA GLOBAL-VARIACIONES RELATIVAS CORRIENTES,Anual,8,ESR-ODA-01,23
9,11434,ESR-ODA-01-9,DEMANDA GLOBAL---DEMANDA INTERNA-EN MILLONES DE LEMPIRAS CONSTANTES,Anual,9,ESR-ODA-01,24


Filtro por correlativo de grupo del indicador:


In [29]:
filtro = 10
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/conteos?formato=Json&correlativogrupo=" * string(filtro)
get_count_gids(key,url)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo,Conteo
,Int64,String,String,String,String,String,Int64
1,47,ESR-PIBT-PROD-DES-CORR-10,Serie Desestacionalizada-VALORES CORRIENTES-11-Propiedad de Vivienda Actividades Inmobiliarias y Empresariales,Trimestral,10,ESR-PIBT-PROD-DES-CORR,96
2,62,ESR-PIBT-PROD-OG-CONST-10,Serie Original-VALORES CONSTANTES-11-Propiedad de Vivienda Actividades Inmobiliarias y Empresariales,Trimestral,10,ESR-PIBT-PROD-OG-CONST,96
3,81,ESR-PIBT-PROD-DES-CONST-10,Serie Desestacionalizada-VALORES CONSTANTES-11-Propiedad de Vivienda Actividades Inmobiliarias y Empresariales,Trimestral,10,ESR-PIBT-PROD-DES-CONST,96
4,94,ESR-PIBT-PROD-OG-CORR-10,Serie Original-VALORES CORRIENTES-11-Propiedad de Vivienda Actividades Inmobiliarias y Empresariales,Trimestral,10,ESR-PIBT-PROD-OG-CORR,96
5,101,ESR-PIBA-01-10,"ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Agricultura, Ganadería, Caza, Silvicultura y Pesca",Anual,10,ESR-PIBA-01,24
6,190,ESR-PIBA-02-10,ESR-PIBA-02 - Precio - Corrientes - Gasto de Consumo Final,Anual,10,ESR-PIBA-02,48
7,208,ESR-PIBA-03-10,ESR-PIBA-03 - Variación - Ingreso Mixto bruto,Anual,10,ESR-PIBA-03,46
8,225,ESR-VAB-01-10,ESR-VAB-01 - Valor - Corrientes - Cría de Ganado Vacuno,Anual,10,ESR-VAB-01,48
9,317,ESR-VAB-02-10,"ESR-VAB-02 - Variación - Constantes - Productos Alimenticios, Bebidas y Tabaco - Elaboración de Productos de Molineria y Alimentos Preparados para Animales",Anual,10,ESR-VAB-02,46


## 8. Conteo de cifras para un indicador

Consulta el conteo de registros disponibles para un indicador en específico. El parámetro "Id" es un campo obligatorio.

El formato para realizar una consulta personalizada es el siguiente:

`https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/conteos[?formato]`


In [30]:
function get_count_id1(key,id)
  url = "https://bchapi-am.azure-api.net/api/v1/indicadores/" * string(id) * "/conteos?formato=Json"
  usr = Dict("clave" => key)
  js = HTTP.get(url; headers = usr)
  df = DataFrames.DataFrame(JSON.parse(String(js.body)))
  df = df[!,[:Id,:Nombre,:Descripcion,:Periodicidad,:CorrelativoGrupo,:Grupo,:Conteo]]
  return df
end

get_count_id1 (generic function with 1 method)

In [31]:
id = 110
get_count_id1(key,id)

Row,Id,Nombre,Descripcion,Periodicidad,CorrelativoGrupo,Grupo,Conteo
,Int64,String,String,String,String,String,Int64
1,110,ESR-PIBA-01-57,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Propiedad de Vivienda,Anual,57,ESR-PIBA-01,24


# Grupos

Las variables están categorizadas en 8 grupos principales y 54 subgrupos (ESE-BP-01 y ESE-CEB-01 contienen variables de periodicidad anual o trimestral, y EM-TPM-01 tiene variables de periodicidad diaria o mensual). La nomenclatura utilizada permite dividir estos en un máximo de cinco niveles, tomando en cuenta el separador (-). Se explicarán los mismos a continuacion, ordenando de acuerdo al número de niveles (número de subgrupos).


In [32]:
df = get_groups(key)
df_groups = @chain df begin
  DataFrames.groupby(
    # [:Grupo,:CorrelativoGrupo,:Periodicidad])
    # [:Grupo])
    [:Grupo,:Periodicidad])
  DataFrames.combine(
    nrow => :Conteo)
end
DataFrames.sort!(df_groups,:Conteo,rev=true)
df_groups = hcat(
  df_groups,
  DataFrames.select(
    df_groups, 
    :Grupo =>
      ByRow(x -> get.(Ref(split(x, '-')), 1:5, missing)) =>
      [:Gr1,:Gr2,:Gr3,:Gr4,:Gr5]))
CSV.write(
    wd .* "/api/grupos_julia.csv",
    delim = ";",
  df_groups);

Todos los grupos pueden clasificarse en al menos dos subgrupos:


In [33]:
df_groups
DataFrames.describe(
  df_groups, 
  :nmissing, 
  length => :length,
  cols = [:Gr1,:Gr2,:Gr3,:Gr4,:Gr5])
DataFrames.transform!(
  df_groups, 
  AsTable(Between(:Gr1,:Gr5)) => ByRow(count_groups) => :Niv_Gr)
DataFrames.sort!(df_groups,[:Grupo,:Niv_Gr],rev=[true,true])

Row,Grupo,Periodicidad,Conteo,Gr1,Gr2,Gr3,Gr4,Gr5,Niv_Gr
,String,String,Int64,SubStrin…,SubStrin…,SubStrin…?,SubStrin…?,SubStrin…?,Int64
1,ESR-VAB-02,Anual,64,ESR,VAB,02,missing,missing,3
2,ESR-VAB-01,Anual,56,ESR,VAB,01,missing,missing,3
3,ESR-PIBT-TDC-Valores,Trimestral,1,ESR,PIBT,TDC,Valores,missing,4
4,ESR-PIBT-PROD-OG-CORR,Trimestral,17,ESR,PIBT,PROD,OG,CORR,5
5,ESR-PIBT-PROD-OG-CONST,Trimestral,17,ESR,PIBT,PROD,OG,CONST,5
6,ESR-PIBT-PROD-DES-CORR,Trimestral,17,ESR,PIBT,PROD,DES,CORR,5
7,ESR-PIBT-PROD-DES-CONST,Trimestral,17,ESR,PIBT,PROD,DES,CONST,5
8,ESR-PIBT-GAST-OG-CORR,Trimestral,7,ESR,PIBT,GAST,OG,CORR,5
9,ESR-PIBT-GAST-OG-CONST,Trimestral,7,ESR,PIBT,GAST,OG,CONST,5


Los 8 grupos principales mencionados permiten consolidar los archivos de acuerdo con el origen de los datos en la página web del BCH, que en su mayoría pueden consultarse en los reportes dinámicos. A su vez, estos grupos pueden subdividirse en 25 subgrupos:

## EOM = Estadísticas de Operaciones Monetarias

### OMA = Operaciones de Mercado Abierto**

Cuatro niveles de agregación:

* 01 = Tasas de Rendimiento Diario de Valores Gubernamentales


In [34]:
filtro = "EOM-OMA-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,701,1,EOM-OMA-01-1,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 1,Diario
2,702,2,EOM-OMA-01-2,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 2,Diario
3,703,3,EOM-OMA-01-3,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 3,Diario
4,704,4,EOM-OMA-01-4,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 4,Diario
5,705,5,EOM-OMA-01-5,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 5,Diario
6,706,6,EOM-OMA-01-6,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 6,Diario
7,707,7,EOM-OMA-01-7,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 7,Diario
8,708,8,EOM-OMA-01-8,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 8,Diario
9,709,9,EOM-OMA-01-9,EOM-OMA-01,Tasas de Rendimiento Diario de Valores Gubernamentales - Dias Plazo - 9,Diario


* 02 = Créditos (Niv_Descr_4 = 1-3) e Inversiones (Niv_Descr_4 = 4-6): Monto, Número de Solicitudes y Tasa


In [35]:
filtro = "EOM-OMA-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6261,1,EOM-OMA-02-1,EOM-OMA-02,EOM-OMA-02-1-Créditos-Monto,Diario
2,6262,2,EOM-OMA-02-2,EOM-OMA-02,EOM-OMA-02-2-Créditos-Número de Solicitudes,Diario
3,6263,3,EOM-OMA-02-3,EOM-OMA-02,EOM-OMA-02-3-Créditos-Tasa,Diario
4,6264,4,EOM-OMA-02-4,EOM-OMA-02,EOM-OMA-02-4-Inversiones-Monto,Diario
5,6265,5,EOM-OMA-02-5,EOM-OMA-02,EOM-OMA-02-5-Inversiones-Número de Solicitudes,Diario
6,6266,6,EOM-OMA-02-6,EOM-OMA-02,EOM-OMA-02-6-Inversiones-Tasa,Diario


* 03 = Monto, Número de Operaciones y Tasa


In [36]:
filtro = "EOM-OMA-03"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6267,1,EOM-OMA-03-1,EOM-OMA-03,EOM-OMA-03-1-Monto,Diario
2,6268,2,EOM-OMA-03-2,EOM-OMA-03,EOM-OMA-03-2-Número de Operaciones,Diario
3,6269,3,EOM-OMA-03-3,EOM-OMA-03,EOM-OMA-03-3-Tasa,Diario


* 04 = Monto, Número de Operaciones y Tasa (Dólares - Lempiras)


In [37]:
filtro = "EOM-OMA-04"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6270,1,EOM-OMA-04-1,EOM-OMA-04,EOM-OMA-04-1-Monto-Dólares,Diario
2,6271,2,EOM-OMA-04-2,EOM-OMA-04,EOM-OMA-04-2-Monto-Lempiras,Diario
3,6272,3,EOM-OMA-04-3,EOM-OMA-04,EOM-OMA-04-3-Número de Operaciones-Dólares,Diario
4,6273,4,EOM-OMA-04-4,EOM-OMA-04,EOM-OMA-04-4-Número de Operaciones-Lempiras,Diario
5,6274,5,EOM-OMA-04-5,EOM-OMA-04,EOM-OMA-04-5-Tasa-Dólares,Diario
6,6275,6,EOM-OMA-04-6,EOM-OMA-04,EOM-OMA-04-6-Tasa-Lempiras,Diario


## ESR = Estadísticas del Sector Real

### COU = Cuadro de Oferta y Utilización

Detalla la interacción del producto con la actividad (el detalle se toma de acuerdo con la estructura de la columna "Nombre"). Para conocer la estructura de las estadísticas derivadas de este cuadro, consultar en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/sector-real/cuentas-nacionales-anuales-base-2000/cuadro-oferta-y-utilizacion).

El orden está de acuerdo con el Nivel de Descripción 2 (Niv_Descr_2) y Nivel de Descripción 3 (Niv_Descr_3) de la columna Nombre, para los casos de:

* VALOR BRUTO DE PRODUCCIÓN: En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).

* CONSUMO INTERMEDIO: En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son las actividades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).

* VALOR AGREGADO: En el cuadro original, las filas son los elementos indispensables para calcular el Valor Agregado Bruto, VAB (Remuneración de los Asalariados, Impuestos sobre la Producción y las Importaciones, Subvenciones, Excedente de Explotación, Ingreso Mixto y Consumo de Capital Fijo) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).

* OFERTA: En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son los componentes de la Oferta Agregada a Precios de Comprador (Producción, Importaciones CIF, Ajuste CIF/FOB, Impuestos y Subvenciones a los Productos y Márgenes de Distribución).

* UTILIZACIÓN: En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son los componentes de la Utilización Agregada a Precios de Comprador (Consumo Intermedio, Exportciones FOB, Gasto de Consumo Final y Formación Bruta de Capital).

Cada uno de estos detalles corresponde a una hoja de Excel en el archivo anual correspondiente.

Tomar en cuenta que, para realizar el cálculo del VAB por NAEH, se debe sumar las columnas de Valor Bruto de Producción y restar las de Consumo Intermedio; adicionalmente, como los datos correspondientes a Impuestos sobre los Productos y los Márgenes de Distribución solamente se encuentran a nivel de NPH, no se puede tener un VAB total que se obtenga por suma de VAB a nivel de actividad individual, en vista que debe restarse las de Subvenciones a los productos, mismas que se encuentran en el Cuadro de Oferta solamente a nivel de NPH. Esta forma de obtener el VAB se conoce como el cálculo del PIB por el enfoque del Ingreso.

Como la Oferta y Utilización están detalladas a nivel de NPH, el VAB por NPH a partir de los dos detalles previos, se puede obtener por dos métodos:

* Suma de Producción + Impuestos - Subvenciones a los Productos + Márgenes de Distribución (Oferta) - Consumo Intermedio (Utilización) => PIB por el enfoque de la Produccion.
* Exportaciones FOB + Gasto de Consumo Final + Formación Bruta de Capital (Utilización) - Importaciones CIF - Ajuste CIF/FOB (Oferta) => PIB por el enfoque del Gasto.


In [38]:
filtro = "ESR-COU-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,7818,1,ESR-COU-01-1,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-ACTIVIDADES INMOBILIARIAS Y EMPRESARIALES,Anual
2,7819,2,ESR-COU-01-2,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-ADMINISTRACIÓN PÚBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACIÓN OBLIGATORIA,Anual
3,7820,3,ESR-COU-01-3,ESR-COU-01,"CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-AGRICULTURA, GANADERIA, CAZA, SILVICULTURA Y PESCA",Anual
4,7821,4,ESR-COU-01-4,ESR-COU-01,"CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-COMERCIO, REPARACIÓN DE VEHÍCULOS AUTOMOTORES, MOTOCICLETAS, EFECTOS PERSONALES Y ENSERES DOMÉSTICOS",Anual
5,7822,5,ESR-COU-01-5,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-COMUNICACIONES,Anual
6,7823,6,ESR-COU-01-6,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-CONSTRUCCIÓN,Anual
7,7824,7,ESR-COU-01-7,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-ELECTRICIDAD Y DISTRIBUCIÓN DE AGUA,Anual
8,7825,8,ESR-COU-01-8,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-EXPLOTACIÓN DE MINAS Y CANTERAS,Anual
9,7826,9,ESR-COU-01-9,ESR-COU-01,CONSUMO INTERMEDIO-ACEITES Y GRASAS ANIMALES Y VEGETALES-HOTELES Y RESTAURANTES,Anual


### CCI = Clasificación Cruzada Industria

Para conocer la estructura de las estadísticas derivadas de este cuadro, consultar en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/sector-real/cuentas-nacionales-anuales-base-2000/clasificacion-cruzada-industria-sector-institucional).

El archivo contiene 3 hojas, que pueden filtrarse de acuerdo con el Nivel de Descripción 1 (Niv_Descr_1) de la variable:

* Producción Bruta
* Consumo Intermedio
* Valor Agregado

Los cuadros en cada hoja contienen los valores a nivel de Industria en las filas y a nivel de Sector Institucional en las columnas.

Debido a que el VAB implica tomar en cuenta impuestos y subvenciones a nivel de NPH, no es posible obtener el cálculo de esta variable por suma a partir de los cuadros del CCI (ver explicación en el apartado del COU).


In [39]:
filtro = "ESR-CCI-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,7447,1,ESR-CCI-01-1,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-Economía Total,Anual
2,7448,2,ESR-CCI-01-2,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-Gobierno General,Anual
3,7449,3,ESR-CCI-01-3,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-Hogares,Anual
4,7450,4,ESR-CCI-01-4,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-ISFLSH,Anual
5,7451,5,ESR-CCI-01-5,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-SIFMI,Anual
6,7452,6,ESR-CCI-01-6,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-Sociedades Financieras,Anual
7,7453,7,ESR-CCI-01-7,ESR-CCI-01,Consumo Intermedio-Actividades Inmobiliarias y Empresariales-Sociedades No Financieras,Anual
8,7454,8,ESR-CCI-01-8,ESR-CCI-01,Consumo Intermedio-Administración Pública y Defensa; Planes de Seguridad Social de Afiliación Obligatoria-Economía Total,Anual
9,7455,9,ESR-CCI-01-9,ESR-CCI-01,Consumo Intermedio-Administración Pública y Defensa; Planes de Seguridad Social de Afiliación Obligatoria-Gobierno General,Anual


### VAB = Valor Agregado Bruto

#### Valor y Variación, Corrientes y Constantes, por NAEH


In [40]:
filtro = "ESR-VAB-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,217,35,ESR-VAB-01-35,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Maíz,Anual
2,218,31,ESR-VAB-01-31,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Fríjol,Anual
3,219,43,ESR-VAB-01-43,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Sorgo,Anual
4,220,20,ESR-VAB-01-20,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Arroz en Granza,Anual
5,221,40,ESR-VAB-01-40,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Palma Africana,Anual
6,222,21,ESR-VAB-01-21,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Banano,Anual
7,223,25,ESR-VAB-01-25,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cultivo de Café,Anual
8,224,48,ESR-VAB-01-48,ESR-VAB-01,"ESR-VAB-01 - Valor - Corrientes - Cultivo de Tubérculos, Hortalizas, Legumbres y Frutas",Anual
9,225,10,ESR-VAB-01-10,ESR-VAB-01,ESR-VAB-01 - Valor - Corrientes - Cría de Ganado Vacuno,Anual


#### Precio y Variación, Corrientes y Constantes, por NAEH


In [41]:
filtro = "ESR-VAB-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,273,57,ESR-VAB-02-57,ESR-VAB-02,"ESR-VAB-02 - Precio - Constantes - Actividad Económica - Productos Alimenticios, Bebidas y Tabaco",Anual
2,274,30,ESR-VAB-02-30,ESR-VAB-02,"ESR-VAB-02 - Precio - Constantes - Actividad Económica - Fabricación de Productos Textiles, Cuero y Calzado",Anual
3,275,50,ESR-VAB-02-50,ESR-VAB-02,ESR-VAB-02 - Precio - Constantes - Actividad Económica - Producción de Madera y Fabricación de Productos de Madera,Anual
4,276,18,ESR-VAB-02-18,ESR-VAB-02,"ESR-VAB-02 - Precio - Constantes - Actividad Económica - Fabricación de Papel y Productos de Papel, Actividades de Edición e Impresión",Anual
5,277,34,ESR-VAB-02-34,ESR-VAB-02,"ESR-VAB-02 - Precio - Constantes - Actividad Económica - Fabricación de Sustancias y Productos Químicos, Derivados del Petróleo, Productos de Caucho y Plásti",Anual
6,278,22,ESR-VAB-02-22,ESR-VAB-02,ESR-VAB-02 - Precio - Constantes - Actividad Económica - Fabricación de Productos de Vidrio y Otros Minerales no Metálicos,Anual
7,279,37,ESR-VAB-02-37,ESR-VAB-02,ESR-VAB-02 - Precio - Constantes - Actividad Económica - Industrias Metálicas Básicas,Anual
8,280,27,ESR-VAB-02-27,ESR-VAB-02,"ESR-VAB-02 - Precio - Constantes - Actividad Económica - Fabricación de Productos Metálicos, Maquinaria y Equipo",Anual
9,281,43,ESR-VAB-02-43,ESR-VAB-02,ESR-VAB-02 - Precio - Constantes - Actividad Económica - Muebles,Anual


Notar que aquí el Nivel de Descripción 6 (Niv_Descr_6) no es una buena opción de desagregación, puesto que los componentes son "Actividad Económica" y “Productos Alimenticios, Bebidas y Tabaco”; esta última está detallada en mayor grado en el Nivel de Descripción 7 (Niv_Descr_7).

### PIBA = PIB Anual

#### Valores Corrientes y Constantes, por Actividad Económica

La información se encuentra en la viñeta "Sector Real" de la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-real).

* Rama de Actividad Económica (RAE)
* Series Agregadas
* Variación RAE
* Variación Series Agregadas


In [42]:
filtro = "ESR-PIBA-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,99,4,ESR-PIBA-01-4,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Actividades Inmobiliarias y Empresariales,Anual
2,100,5,ESR-PIBA-01-5,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Administración Pública y Defensa; Planes de Seguridad Social de Afiliación Obligatoria,Anual
3,101,10,ESR-PIBA-01-10,ESR-PIBA-01,"ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Agricultura, Ganadería, Caza, Silvicultura y Pesca",Anual
4,102,13,ESR-PIBA-01-13,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Comercio,Anual
5,103,17,ESR-PIBA-01-17,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Comunicaciones,Anual
6,104,21,ESR-PIBA-01-21,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Construcción,Anual
7,105,25,ESR-PIBA-01-25,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Electricidad y Distribución de Agua,Anual
8,106,32,ESR-PIBA-01-32,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Explotación de Minas y Canteras,Anual
9,107,34,ESR-PIBA-01-34,ESR-PIBA-01,ESR-PIBA-01 - Constantes - Rama de Actividad Económica - Hoteles y Restaurantes,Anual


#### Precios y Variación, por Componente del Gasto


In [43]:
filtro = "ESR-PIBA-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,183,2,ESR-PIBA-02-2,ESR-PIBA-02,ESR-PIBA-02 - Precio - Constantes - Exportación fob de bienes y servicios,Anual
2,184,6,ESR-PIBA-02-6,ESR-PIBA-02,ESR-PIBA-02 - Precio - Constantes - Formación Bruta de Capital Fijo,Anual
3,185,12,ESR-PIBA-02-12,ESR-PIBA-02,ESR-PIBA-02 - Precio - Constantes - Gasto de Consumo Final,Anual
4,186,13,ESR-PIBA-02-13,ESR-PIBA-02,ESR-PIBA-02 - Precio - Constantes - Importación cif de bienes y servicios,Anual
5,187,18,ESR-PIBA-02-18,ESR-PIBA-02,ESR-PIBA-02 - Precio - Constantes - Variación de Existencias,Anual
6,188,3,ESR-PIBA-02-3,ESR-PIBA-02,ESR-PIBA-02 - Precio - Corrientes - Exportación fob de bienes y servicios,Anual
7,189,7,ESR-PIBA-02-7,ESR-PIBA-02,ESR-PIBA-02 - Precio - Corrientes - Formación Bruta de Capital Fijo,Anual
8,190,10,ESR-PIBA-02-10,ESR-PIBA-02,ESR-PIBA-02 - Precio - Corrientes - Gasto de Consumo Final,Anual
9,191,14,ESR-PIBA-02-14,ESR-PIBA-02,ESR-PIBA-02 - Precio - Corrientes - Importación cif de bienes y servicios,Anual


Tipo de Valores ("Niv_Descr_5"):

* Corrientes
* Constantes

#### Precios y Variación, por Componente del Ingreso


In [44]:
filtro = "ESR-PIBA-03"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,201,4,ESR-PIBA-03-4,ESR-PIBA-03,ESR-PIBA-03 - Precio - Excedente de Explotación bruto,Anual
2,202,3,ESR-PIBA-03-3,ESR-PIBA-03,ESR-PIBA-03 - Precio - Impuestos netos de subvenciones sobre la producción y las importaciones,Anual
3,203,5,ESR-PIBA-03-5,ESR-PIBA-03,ESR-PIBA-03 - Precio - Ingreso Mixto bruto,Anual
4,204,6,ESR-PIBA-03-6,ESR-PIBA-03,ESR-PIBA-03 - Precio - Producto Interno Bruto (pm),Anual
5,205,1,ESR-PIBA-03-1,ESR-PIBA-03,ESR-PIBA-03 - Precio - Remuneración de los asalariados,Anual
6,206,9,ESR-PIBA-03-9,ESR-PIBA-03,ESR-PIBA-03 - Variación - Excedente de Explotación bruto,Anual
7,207,8,ESR-PIBA-03-8,ESR-PIBA-03,ESR-PIBA-03 - Variación - Impuestos netos de subvenciones sobre la producción y las importaciones,Anual
8,208,10,ESR-PIBA-03-10,ESR-PIBA-03,ESR-PIBA-03 - Variación - Ingreso Mixto bruto,Anual
9,209,2,ESR-PIBA-03-2,ESR-PIBA-03,ESR-PIBA-03 - Variación - Producto Interno Bruto (pm),Anual


Componente del Ingreso:

* Remuneración de los asalariados
* Impuestos netos de subvenciones sobre la producción y las importaciones
* Excedente de Explotación bruto
* Ingreso Mixto bruto
* Producto Interno Bruto (pm)

#### Ingreso Nacional, PIB, PNB


In [45]:
filtro = "ESR-PIBA-04"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,211,2,ESR-PIBA-04-2,ESR-PIBA-04,ESR-PIBA-04 - Dólares - IngresoNacionalDolares - Ingreso Nacional,Anual
2,212,4,ESR-PIBA-04-4,ESR-PIBA-04,ESR-PIBA-04 - Dólares - ProductoInternoBrutoDolares - Producto Interno Bruto,Anual
3,213,6,ESR-PIBA-04-6,ESR-PIBA-04,ESR-PIBA-04 - Dólares - ProductoNacionalBrutoDolares - Producto Nacional Bruto,Anual
4,214,1,ESR-PIBA-04-1,ESR-PIBA-04,ESR-PIBA-04 - Lempiras - IngresoNacional - Ingreso Nacional,Anual
5,215,3,ESR-PIBA-04-3,ESR-PIBA-04,ESR-PIBA-04 - Lempiras - ProductoInternoBruto - Producto Interno Bruto,Anual
6,216,5,ESR-PIBA-04-5,ESR-PIBA-04,ESR-PIBA-04 - Lempiras - ProductoNacionalBruto - Producto Nacional Bruto,Anual


* Dólares
* Lempiras

### PIBT = PIB Trimestral

Lo que se detalla a continuación se obtiene a partir de los datos del [Portal de Consulta del Sistema de Cuentas Trimestrales](https://sisee.bch.hn/SCNT/Login.aspx?ReturnUrl=%2fSCNT%2fPortalPrincipal.aspx).

#### PROD = PIB Trimestral por el Enfoque de la Producción

CorrelativoGrupo:

1. Agricultura Ganaderí­a Caza Silvicultura y Pesca 
2. Explotación de Minas y Canteras
3. Industrias Manufactureras
4. Electricidad y Distribución de Agua
5. Construcción
6. Comercio Reparación de Vehí­culos Automotores Motocicletas Efectos Personales y Enseres Domésticos Hoteles y Restaurantes
7. Transporte y Almacenamiento
8. Comunicaciones
9. Intermediación Financiera
10. Propiedad de Vivienda Actividades Inmobiliarias y Empresariales
11. Administración Pública y Defensa Planes de Seguridad Social de Afiliación Obligatoria
12. Servicios de Enseñanza Servicios Sociales y de Salud
13. Servicios Comunitarios Sociales y Personales
14. Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI)
15. Valor Agregado Bruto a precios básicos
16. Más Impuestos netos de subvenciones sobre la producción y las importaciones
17. Producto Interno Bruto a precios de mercado


##### OG-CONST = PIB Trimestral por el Enfoque de la Producción, Serie Original, Valores Constantes


In [46]:
filtro = "ESR-PIBT-PROD-OG-CONST"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,50,12,ESR-PIBT-PROD-OG-CONST-12,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,Trimestral
2,55,17,ESR-PIBT-PROD-OG-CONST-17,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-23-Producto Interno Bruto a precios de mercado,Trimestral
3,56,16,ESR-PIBT-PROD-OG-CONST-16,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-22-Más Impuestos netos de subvenciones sobre la producción y las importaciones,Trimestral
4,57,15,ESR-PIBT-PROD-OG-CONST-15,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,Trimestral
5,58,14,ESR-PIBT-PROD-OG-CONST-14,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-20-Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI),Trimestral
6,59,13,ESR-PIBT-PROD-OG-CONST-13,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-16-Servicios Comunitarios Sociales y Personales,Trimestral
7,60,11,ESR-PIBT-PROD-OG-CONST-11,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-13-Administración Pública y Defensa Planes de Seguridad Social de Afiliación Obligatoria,Trimestral
8,62,10,ESR-PIBT-PROD-OG-CONST-10,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-11-Propiedad de Vivienda Actividades Inmobiliarias y Empresariales,Trimestral
9,63,9,ESR-PIBT-PROD-OG-CONST-9,ESR-PIBT-PROD-OG-CONST,Serie Original-VALORES CONSTANTES-10-Intermediación Financiera,Trimestral


##### OG-CORR = PIB Trimestral por el Enfoque de la Producción, Serie Original, Valores Corrientes


In [47]:
filtro = "ESR-PIBT-PROD-OG-CORR"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,23,8,ESR-PIBT-PROD-OG-CORR-8,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-9-Comunicaciones,Trimestral
2,51,4,ESR-PIBT-PROD-OG-CORR-4,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-4-Electricidad y Distribución de Agua,Trimestral
3,52,3,ESR-PIBT-PROD-OG-CORR-3,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-3-Industrias Manufactureras,Trimestral
4,53,2,ESR-PIBT-PROD-OG-CORR-2,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-2-Explotación de Minas y Canteras,Trimestral
5,54,1,ESR-PIBT-PROD-OG-CORR-1,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-1-Agricultura Ganadería Caza Silvicultura y Pesca,Trimestral
6,61,6,ESR-PIBT-PROD-OG-CORR-6,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-6-Comercio Reparación de Vehículos Automotores Motocicletas Efectos Personales y Enseres Domésticos Hoteles y Restaurantes,Trimestral
7,73,7,ESR-PIBT-PROD-OG-CORR-7,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-8-Transporte y Almacenamiento,Trimestral
8,75,5,ESR-PIBT-PROD-OG-CORR-5,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-5-Construcción,Trimestral
9,87,17,ESR-PIBT-PROD-OG-CORR-17,ESR-PIBT-PROD-OG-CORR,Serie Original-VALORES CORRIENTES-23-Producto Interno Bruto a precios de mercado,Trimestral


##### DES-CONST = PIB Trimestral por el Enfoque de la Producción, Serie Desestacionalizada, Valores Constantes


In [48]:
filtro = "ESR-PIBT-PROD-DES-CONST"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,1,15,ESR-PIBT-PROD-DES-CONST-15,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,Trimestral
2,38,17,ESR-PIBT-PROD-DES-CONST-17,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-23-Producto Interno Bruto a precios de mercado,Trimestral
3,48,16,ESR-PIBT-PROD-DES-CONST-16,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-22-Más Impuestos netos de subvenciones sobre la producción y las importaciones,Trimestral
4,49,14,ESR-PIBT-PROD-DES-CONST-14,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-20-Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI),Trimestral
5,74,1,ESR-PIBT-PROD-DES-CONST-1,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-1-Agricultura Ganadería Caza Silvicultura y Pesca,Trimestral
6,76,5,ESR-PIBT-PROD-DES-CONST-5,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-5-Construcción,Trimestral
7,77,6,ESR-PIBT-PROD-DES-CONST-6,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-6-Comercio Reparación de Vehículos Automotores Motocicletas Efectos Personales y Enseres Domésticos Hoteles y Restaurantes,Trimestral
8,78,7,ESR-PIBT-PROD-DES-CONST-7,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-8-Transporte y Almacenamiento,Trimestral
9,79,8,ESR-PIBT-PROD-DES-CONST-8,ESR-PIBT-PROD-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-9-Comunicaciones,Trimestral


##### DES-CORR = PIB Trimestral por el Enfoque de la Producción, Serie Desestacionalizada, Valores Corrientes


In [49]:
filtro = "ESR-PIBT-PROD-DES-CORR"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,2,12,ESR-PIBT-PROD-DES-CORR-12,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,Trimestral
2,3,13,ESR-PIBT-PROD-DES-CORR-13,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-16-Servicios Comunitarios Sociales y Personales,Trimestral
3,4,14,ESR-PIBT-PROD-DES-CORR-14,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-20-Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI),Trimestral
4,5,15,ESR-PIBT-PROD-DES-CORR-15,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,Trimestral
5,6,16,ESR-PIBT-PROD-DES-CORR-16,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-22-Más Impuestos netos de subvenciones sobre la producción y las importaciones,Trimestral
6,7,17,ESR-PIBT-PROD-DES-CORR-17,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-23-Producto Interno Bruto a precios de mercado,Trimestral
7,13,9,ESR-PIBT-PROD-DES-CORR-9,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-10-Intermediación Financiera,Trimestral
8,14,8,ESR-PIBT-PROD-DES-CORR-8,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-9-Comunicaciones,Trimestral
9,15,7,ESR-PIBT-PROD-DES-CORR-7,ESR-PIBT-PROD-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-8-Transporte y Almacenamiento,Trimestral


#### GAST = PIB Trimestral por el Enfoque del Gasto

CorrelativoGrupo:

1.	Gastos de Consumo Final
2.	Sector Privado
3.	Sector PÃºblico
4.	FormaciÃ³n Bruta de Capital
5.	Exportaciones de Bienes y Servicios
6.	ImportaciÃ³n de Bienes y Servicios
7.	Producto Interno Bruto a precios de mercado


##### OG-CONST = PIB Trimestral por el Enfoque de del Gasto, Serie Original, Valores Constantes


In [50]:
filtro = "ESR-PIBT-GAST-OG-CONST"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,8,1,ESR-PIBT-GAST-OG-CONST-1,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-1-Gastos de Consumo Final,Trimestral
2,9,2,ESR-PIBT-GAST-OG-CONST-2,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-2-Sector Privado,Trimestral
3,10,3,ESR-PIBT-GAST-OG-CONST-3,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-3-Sector Público,Trimestral
4,11,5,ESR-PIBT-GAST-OG-CONST-5,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-5-Exportaciones de Bienes y Servicios,Trimestral
5,12,4,ESR-PIBT-GAST-OG-CONST-4,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-4-Formación Bruta de Capital,Trimestral
6,24,6,ESR-PIBT-GAST-OG-CONST-6,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-6-Importación de Bienes y Servicios,Trimestral
7,72,7,ESR-PIBT-GAST-OG-CONST-7,ESR-PIBT-GAST-OG-CONST,Serie Original-VALORES CONSTANTES-7-Producto Interno Bruto a precios de mercado,Trimestral


##### OG-CORR = PIB Trimestral por el Enfoque de del Gasto, Serie Original, Valores Corrientes


In [51]:
filtro = "ESR-PIBT-GAST-OG-CORR"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,37,1,ESR-PIBT-GAST-OG-CORR-1,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-1-Gastos de Consumo Final,Trimestral
2,41,7,ESR-PIBT-GAST-OG-CORR-7,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-7-Producto Interno Bruto a precios de mercado,Trimestral
3,42,6,ESR-PIBT-GAST-OG-CORR-6,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-6-Importación de Bienes y Servicios,Trimestral
4,43,5,ESR-PIBT-GAST-OG-CORR-5,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-5-Exportaciones de Bienes y Servicios,Trimestral
5,44,4,ESR-PIBT-GAST-OG-CORR-4,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-4-Formación Bruta de Capital,Trimestral
6,45,3,ESR-PIBT-GAST-OG-CORR-3,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-3-Sector Público,Trimestral
7,46,2,ESR-PIBT-GAST-OG-CORR-2,ESR-PIBT-GAST-OG-CORR,Serie Original-VALORES CORRIENTES-2-Sector Privado,Trimestral


##### DES-CONST = PIB Trimestral por el Enfoque del Gasto, Serie Desestacionalizada, Valores Constantes


In [52]:
filtro = "ESR-PIBT-GAST-DES-CONST"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,26,5,ESR-PIBT-GAST-DES-CONST-5,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-5-Exportaciones de Bienes y Servicios,Trimestral
2,27,6,ESR-PIBT-GAST-DES-CONST-6,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-6-Importación de Bienes y Servicios,Trimestral
3,28,7,ESR-PIBT-GAST-DES-CONST-7,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-7-Producto Interno Bruto a precios de mercado,Trimestral
4,35,4,ESR-PIBT-GAST-DES-CONST-4,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-4-Formación Bruta de Capital,Trimestral
5,36,3,ESR-PIBT-GAST-DES-CONST-3,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-3-Sector Público,Trimestral
6,39,2,ESR-PIBT-GAST-DES-CONST-2,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-2-Sector Privado,Trimestral
7,40,1,ESR-PIBT-GAST-DES-CONST-1,ESR-PIBT-GAST-DES-CONST,Serie Desestacionalizada-VALORES CONSTANTES-1-Gastos de Consumo Final,Trimestral


##### DES-CORR = PIB Trimestral por el Enfoque del Gasto, Serie Desestacionalizada, Valores Corrientes


In [53]:
filtro = "ESR-PIBT-GAST-DES-CORR"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,25,7,ESR-PIBT-GAST-DES-CORR-7,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-7-Producto Interno Bruto a precios de mercado,Trimestral
2,29,1,ESR-PIBT-GAST-DES-CORR-1,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-1-Gastos de Consumo Final,Trimestral
3,30,2,ESR-PIBT-GAST-DES-CORR-2,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-2-Sector Privado,Trimestral
4,31,3,ESR-PIBT-GAST-DES-CORR-3,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-3-Sector Público,Trimestral
5,32,4,ESR-PIBT-GAST-DES-CORR-4,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-4-Formación Bruta de Capital,Trimestral
6,33,5,ESR-PIBT-GAST-DES-CORR-5,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-5-Exportaciones de Bienes y Servicios,Trimestral
7,34,6,ESR-PIBT-GAST-DES-CORR-6,ESR-PIBT-GAST-DES-CORR,Serie Desestacionalizada-VALORES CORRIENTES-6-Importación de Bienes y Servicios,Trimestral


#### TDC = PIB Trimestral, Tasas de Crecimiento, Valores Ajustados


In [54]:
filtro = "ESR-PIBT-TDC-Valores"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,98,01,ESR-PIBT-TDC-Valores-01,ESR-PIBT-TDC-Valores,Valores Ajustados,Trimestral


### ODA = Oferta y Demanda Agregada

La estructura de estas variables puede verse en el [INE](https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fine.gob.hn%2Fv4%2Fwp-content%2Fuploads%2F2023%2F06%2F7.1.5-Oferta-y-demanda-global-millones-de-lempiras-corrientes-2017-2021.xlsx&wdOrigin=BROWSELINK).

#### Oferta Global

Los números corresponden al correlativo de acuerdo con el cuadro citado.

**Niv_Descr_2 = VALOR AGREGADO BRUTO; Niv_Descr_3:**

3. AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y PESCA (A PRECIOS BÁSICOS)
4. EXPLOTACIÓN DE MINAS Y CANTERAS (A PRECIOS BÁSICOS)
5. INDUSTRIA MANUFACTURERA (A PRECIOS BÁSICOS)
6. CONSTRUCCIÓN (A PRECIOS BÁSICOS)
7. COMERCIO, HOTELES Y RESTAURANTES (A PRECIOS BÁSICOS)
9. MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)

Niv_Descr_4 corresponde a:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

**Niv_Descr_2 = ""; Niv_Descr_3 = ""; Niv_Descr_4:**

8. OTROS SERVICIOS (A PRECIOS BÁSICOS)
10. IMPUESTOS NETOS DE SUBVENCIONES SOBRE LA PRODUCCIÓN Y LAS IMPORTACIONES
11. PRODUCTO INTERNO BRUTO A PRECIOS DE MERCADO
12. IMPORTACIONES DE BIENES Y SERVICIOS C.I.F.
1. OFERTA GLOBAL
13. TOTAL VALOR AGREGADO BRUTO A PRECIOS BáSICOS
14. VALOR AGREGADO BRUTO
2. VALOR AGREGADO BRUTO A PRECIOS BÁSICOS

Niv_Descr_5 corresponde a:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

#### Demanda Global

**Niv_Descr_2 = ""; Niv_Descr_3 = ""; Niv_Descr_4:**

1. DEMANDA GLOBAL
2. DEMANDA INTERNA
3. GASTOS DE CONSUMO FINAL
4. INVERSIÓN INTERNA BRUTA
5. FORMACIÓN BRUTA DE CAPITAL FIJO
6. VARIACIÓN DE EXISTENCIAS
7. DEMANDA EXTERNA
8. EXPORTACIÓN DE BIENES Y SERVICIOS F.O.B.

Niv_Descr_5, de acuerdo con lo anterior, es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

**Variables con valores en Niv_Descr_2 y Niv_Descr_3, corresponde a las siguientes series:**

3. GASTOS DE CONSUMO FINAL
5. FORMACIÓN BRUTA DE CAPITAL FIJO

De Niv_Descr_3 se obtiene:

a. SECTOR PRIVADO
b. SECTOR PÚBLICO

Niv_Descr_4, de acuerdo con lo anterior, es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES


In [55]:
filtro = "ESR-ODA-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,11426,1,ESR-ODA-01-1,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA EXTERNA-EN MILLONES DE LEMPIRAS CONSTANTES,Anual
2,11427,2,ESR-ODA-01-2,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA EXTERNA-EN MILLONES DE LEMPIRAS CORRIENTES,Anual
3,11428,3,ESR-ODA-01-3,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA EXTERNA-VARIACIONES RELATIVAS CONSTANTE,Anual
4,11429,4,ESR-ODA-01-4,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA EXTERNA-VARIACIONES RELATIVAS CORRIENTES,Anual
5,11430,5,ESR-ODA-01-5,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA GLOBAL-EN MILLONES DE LEMPIRAS CONSTANTES,Anual
6,11431,6,ESR-ODA-01-6,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA GLOBAL-EN MILLONES DE LEMPIRAS CORRIENTES,Anual
7,11432,7,ESR-ODA-01-7,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA GLOBAL-VARIACIONES RELATIVAS CONSTANTE,Anual
8,11433,8,ESR-ODA-01-8,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA GLOBAL-VARIACIONES RELATIVAS CORRIENTES,Anual
9,11434,9,ESR-ODA-01-9,ESR-ODA-01,DEMANDA GLOBAL---DEMANDA INTERNA-EN MILLONES DE LEMPIRAS CONSTANTES,Anual


### IMAE = Índice Mensual de la Actividad Económica

Las actividades (Niv_Descr_4) son:

1. Agricultura, Ganaderia, Silvicultura y Pesca
2. Minas y Canteras
3. Industria Manufacturera
4. Electricidad y Agua
5. Construcción
6. Comercio
7. Hoteles y Restaurantes
8. Transporte y Almacenamiento
9. Correo y Telecomunicaciones
10. Inter. Financiera, Seguros y Fondos de Pensiones
11. Otros Servicios
12. Serie Original
13. Tendencia Ciclo

El tipo de valor (Niv_Descr_5) de dichas variables es:

* Índice
* Acumulada
* Interanual


In [56]:
filtro = "ESR-IMAE-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,337,4,ESR-IMAE-01-4,ESR-IMAE-01,"ESR-IMAE-01 - Agricultura, Ganaderia, Silvicultura y Pesca - Acumulada",Mensual
2,338,16,ESR-IMAE-01-16,ESR-IMAE-01,"ESR-IMAE-01 - Agricultura, Ganaderia, Silvicultura y Pesca - Interanual",Mensual
3,339,1,ESR-IMAE-01-1,ESR-IMAE-01,"ESR-IMAE-01 - Agricultura, Ganaderia, Silvicultura y Pesca - Índice",Mensual
4,340,5,ESR-IMAE-01-5,ESR-IMAE-01,ESR-IMAE-01 - Minas y Canteras - Acumulada,Mensual
5,341,17,ESR-IMAE-01-17,ESR-IMAE-01,ESR-IMAE-01 - Minas y Canteras - Interanual,Mensual
6,342,12,ESR-IMAE-01-12,ESR-IMAE-01,ESR-IMAE-01 - Minas y Canteras - Índice,Mensual
7,343,6,ESR-IMAE-01-6,ESR-IMAE-01,ESR-IMAE-01 - Industria Manufacturera - Acumulada,Mensual
8,344,18,ESR-IMAE-01-18,ESR-IMAE-01,ESR-IMAE-01 - Industria Manufacturera - Interanual,Mensual
9,345,23,ESR-IMAE-01-23,ESR-IMAE-01,ESR-IMAE-01 - Industria Manufacturera - Índice,Mensual


## ESE = Estadísticas del Sector Externo

### DE = Deuda Externa

#### 01 - Deudor


In [57]:
filtro = "ESE-DE-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6710,1,ESE-DE-01-1,ESE-DE-01,Deudor-Autoridad Monetaria,Mensual
2,6711,2,ESE-DE-01-2,ESE-DE-01,Deudor-Empresas Públicas no Financieras,Mensual
3,6712,3,ESE-DE-01-3,ESE-DE-01,Deudor-Gobierno General,Mensual
4,6713,4,ESE-DE-01-4,ESE-DE-01,Deudor-Instituciones Públicas Financieras,Mensual


#### 02 - Acreedor


In [58]:
filtro = "ESE-DE-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6714,1,ESE-DE-02-1,ESE-DE-02,Acreedor-Sector Privado Bilateral,Trimestral
2,6715,2,ESE-DE-02-2,ESE-DE-02,Acreedor-Sector Público Bilateral,Trimestral
3,6716,3,ESE-DE-02-3,ESE-DE-02,Acreedor-Sector Público Comercial,Trimestral
4,6717,4,ESE-DE-02-4,ESE-DE-02,Acreedor-Sector Privado Comercial,Trimestral
5,6718,5,ESE-DE-02-5,ESE-DE-02,Acreedor-Sector Público Multilateral,Trimestral
6,6719,6,ESE-DE-02-6,ESE-DE-02,Acreedor-Sector Privado Multilateral,Trimestral


### 03 - Institución

Las instituciones son:

* Banco Central
* Gobierno general
* Otras sociedades financieras
* Otros Sectores
* Saldo bruto de la deuda externa
* Sociedades no financieras

Las cuentas son: 

* Asignación de Derechos especiales de giro
* Créditos y anticipos comerciales
* Moneda y depósitos
* Otros pasivos de deuda
* Préstamos
* Tí­tulos de deuda
* Saldo bruto de la deuda externa


In [59]:
filtro = "ESE-DE-03"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6720,1,ESE-DE-03-1,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-0-3 meses,Trimestral
2,6721,2,ESE-DE-03-2,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-12-18 meses,Trimestral
3,6722,3,ESE-DE-03-3,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-18-24 meses,Trimestral
4,6723,4,ESE-DE-03-4,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-2 años en adelante,Trimestral
5,6724,5,ESE-DE-03-5,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-3-6 meses,Trimestral
6,6725,6,ESE-DE-03-6,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-6-9 meses,Trimestral
7,6726,7,ESE-DE-03-7,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-9-12 meses,Trimestral
8,6727,8,ESE-DE-03-8,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro-De pago inmediato,Trimestral
9,6728,9,ESE-DE-03-9,ESE-DE-03,Banco Central-Asignación de Derechos especiales de giro - Intereses-0-3 meses,Trimestral


### 04 - Pagos que vencen en un año o menos


In [60]:
filtro = "ESE-DE-04"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,7392,1,ESE-DE-04-1,ESE-DE-04,Gobierno general-A corto plazo según el vencimiento original-Asignación de Derechos especiales de giro-Pagos que vencen en un año o menos,Trimestral
2,7393,2,ESE-DE-04-2,ESE-DE-04,Gobierno general-A corto plazo según el vencimiento original-Moneda y depósitos-Pagos que vencen en un año o menos,Trimestral
3,7394,3,ESE-DE-04-3,ESE-DE-04,Gobierno general-A corto plazo según el vencimiento original-Títulos de deuda-Pagos que vencen en un año o menos,Trimestral
4,7395,4,ESE-DE-04-4,ESE-DE-04,Gobierno general-A corto plazo según el vencimiento original-Préstamos-Pagos que vencen en un año o menos,Trimestral
5,7396,5,ESE-DE-04-5,ESE-DE-04,Gobierno general-A corto plazo según el vencimiento original-Créditos y anticipos comerciales-Pagos que vencen en un año o menos,Trimestral
6,7397,6,ESE-DE-04-6,ESE-DE-04,Gobierno general-A corto plazo según el vencimiento original-Otros pasivos de deuda-Pagos que vencen en un año o menos,Trimestral
7,7398,7,ESE-DE-04-7,ESE-DE-04,Gobierno general-A largo plazo exigible dentro de un año o antes-Asignación de Derechos especiales de giro-Pagos que vencen en un año o menos,Trimestral
8,7399,8,ESE-DE-04-8,ESE-DE-04,Gobierno general-A largo plazo exigible dentro de un año o antes-Moneda y depósitos-Pagos que vencen en un año o menos,Trimestral
9,7400,9,ESE-DE-04-9,ESE-DE-04,Gobierno general-A largo plazo exigible dentro de un año o antes-Títulos de deuda-Pagos que vencen en un año o menos,Trimestral


Por institución:

* Gobierno general
* Banco Central
* Otros Sectores
* Otras sociedades financieras
* Sociedades no financieras
* Deuda externa total a corto plazo

Por plazos:

* A corto plazo según el vencimiento original
* A largo plazo exigible dentro de un año o antes
* Deuda externa total a corto plazo

Por Cuenta:

* Asignación de Derechos especiales de giro
* Moneda y depósitos
* Tí­tulos de deuda
* Préstamos
* Créditos y anticipos comerciales
* Otros pasivos de deuda
* Deuda externa total a corto plazo (según el vencimiento residual)

**OJO: REVISAR LA CONSULTA A PARTIR DE 7392 HASTA 7446**

### CEB = Comercio Exterior de Bienes

#### 01 - Productos

* Banano
* Café
* Aceite de Palma
* Camarones
* Melones y Sandí­as
* Legumbres y Hortalizas
* Puros o Cigarros
* Textiles
* Jabones
* Plásticos y su Manufacturas
* Papel y Cartón

Tipo de valor:

* Volumen
* Valor
* Precio

Periodicidad:

* Trimestral
* Anual


In [61]:
filtro = "ESE-CEB-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,414,1,ESE-CEB-01-1,ESE-CEB-01,ESE-CEB-01-1-Banano-Valor-Trimestral-Trimestral,Trimestral
2,415,2,ESE-CEB-01-2,ESE-CEB-01,ESE-CEB-01-2-Banano-Valor-Anual-Anual,Anual
3,416,3,ESE-CEB-01-3,ESE-CEB-01,ESE-CEB-01-3-Banano-Volumen-Anual-Anual,Anual
4,417,4,ESE-CEB-01-4,ESE-CEB-01,ESE-CEB-01-4-Banano-Volumen-Trimestral-Trimestral,Trimestral
5,418,5,ESE-CEB-01-5,ESE-CEB-01,ESE-CEB-01-5-Banano-Precio-Anual-Anual,Anual
6,419,6,ESE-CEB-01-6,ESE-CEB-01,ESE-CEB-01-6-Banano-Precio-Trimestral-Trimestral,Trimestral
7,420,7,ESE-CEB-01-7,ESE-CEB-01,ESE-CEB-01-7-Café-Valor-Anual-Anual,Anual
8,421,8,ESE-CEB-01-8,ESE-CEB-01,ESE-CEB-01-8-Café-Valor-Trimestral-Trimestral,Trimestral
9,422,9,ESE-CEB-01-9,ESE-CEB-01,ESE-CEB-01-9-Café-Volumen-Anual-Anual,Anual


**OJO: Niv_Descr_7 y Niv_Descr_8 son iguales**

#### 02 - Combustibles

* Gasolina regular
* Bunker
* Diesel
* Total combustibles
* Gasolina superior
* Kerosene
* LPG

Tipo de valor:

* Barriles
* Valor
* Precio


In [62]:
filtro = "ESE-CEB-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,497,1,ESE-CEB-02-1,ESE-CEB-02,ESE-CEB-02-1-Gasolina regular-Barriles,Mensual
2,498,10,ESE-CEB-02-10,ESE-CEB-02,ESE-CEB-02-10-Bunker-Precio,Mensual
3,499,11,ESE-CEB-02-11,ESE-CEB-02,ESE-CEB-02-11-Gasolina regular-Precio,Mensual
4,500,12,ESE-CEB-02-12,ESE-CEB-02,ESE-CEB-02-12-Diésel-Precio,Mensual
5,501,13,ESE-CEB-02-13,ESE-CEB-02,ESE-CEB-02-13-Total combustibles-Precio,Mensual
6,502,14,ESE-CEB-02-14,ESE-CEB-02,ESE-CEB-02-14-Gasolina superior-Precio,Mensual
7,503,15,ESE-CEB-02-15,ESE-CEB-02,ESE-CEB-02-15-Kerosene-Valor,Mensual
8,504,16,ESE-CEB-02-16,ESE-CEB-02,ESE-CEB-02-16-Total combustibles-Valor,Mensual
9,505,17,ESE-CEB-02-17,ESE-CEB-02,ESE-CEB-02-17-LPG-Valor,Mensual


#### 03 AE, BT Sección, Secciones

* Importaciones AE
* Importaciones BT Sección
* Importaciones Secciones: Valores en Millones de USD y Peso en Kilos


In [63]:
filtro = "ESE-CEB-03"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6458,1,ESE-CEB-03-1,ESE-CEB-03,Importaciones AE - 1. Alimentos y bebidas,Trimestral
2,6459,2,ESE-CEB-03-2,ESE-CEB-03,Importaciones AE - 1.1 Sector Primario,Trimestral
3,6460,3,ESE-CEB-03-3,ESE-CEB-03,Importaciones AE - 1.1.1 Principalmente para la industria,Trimestral
4,6461,4,ESE-CEB-03-4,ESE-CEB-03,Importaciones AE - 1.1.2 Principalmente para consumo doméstico,Trimestral
5,6462,5,ESE-CEB-03-5,ESE-CEB-03,Importaciones AE - 1.2 Procesados,Trimestral
6,6463,6,ESE-CEB-03-6,ESE-CEB-03,Importaciones AE - 1.2.1 Principalmente para la industria,Trimestral
7,6464,7,ESE-CEB-03-7,ESE-CEB-03,Importaciones AE - 1.2.2 Principalmente para consumo doméstico,Trimestral
8,6465,8,ESE-CEB-03-8,ESE-CEB-03,Importaciones AE - 2. Suministros industriales no especificados en otra parte,Trimestral
9,6466,9,ESE-CEB-03-9,ESE-CEB-03,Importaciones AE - 2.1 Sector Primario,Trimestral


#### 04 AE y BT

* Exportaciones AE
* Exportaciones BT


In [64]:
filtro = "ESE-CEB-04"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6551,1,ESE-CEB-04-1,ESE-CEB-04,"Exportaciones AE- Producto - 1 Agricultura, Ganadería, Silvicultura y Pesca",Trimestral
2,6552,2,ESE-CEB-04-2,ESE-CEB-04,Exportaciones AE- Producto - 1.1 Banano,Trimestral
3,6553,3,ESE-CEB-04-3,ESE-CEB-04,Exportaciones AE- Producto - 1.2 Melones y Sandías,Trimestral
4,6554,4,ESE-CEB-04-4,ESE-CEB-04,"Exportaciones AE- Producto - 1.3 Otras hortalizas, legumbres y demás vegetales y semillas de hortalizas",Trimestral
5,6555,5,ESE-CEB-04-5,ESE-CEB-04,Exportaciones AE- Producto - 1.4 Hortalizas y legumbres orientales,Trimestral
6,6556,6,ESE-CEB-04-6,ESE-CEB-04,Exportaciones AE- Producto - 1.5 Piña,Trimestral
7,6557,7,ESE-CEB-04-7,ESE-CEB-04,Exportaciones AE- Producto - 1.6 Tubérculos y vegetales de raíz,Trimestral
8,6558,8,ESE-CEB-04-8,ESE-CEB-04,"Exportaciones AE- Producto - 1.7 Plantas vivas, flores, capullos cortados y semillas de flores",Trimestral
9,6559,9,ESE-CEB-04-9,ESE-CEB-04,"Exportaciones AE- Producto - 1.8 Otros cultivos, n.c.p.",Trimestral


### BP = Balanza de Pagos

- Trimestral
- Anual


In [65]:
filtro = "ESE-BP-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,6276,1,ESE-BP-01-1,ESE-BP-01,CUENTA CORRIENTE - Mensual,Trimestral
2,6277,2,ESE-BP-01-2,ESE-BP-01,BALANZA DE BIENES - Mensual,Trimestral
3,6278,3,ESE-BP-01-3,ESE-BP-01,1. Exportaciones FOB - Mensual,Trimestral
4,6279,4,ESE-BP-01-4,ESE-BP-01,1.1 Mercancías Generales - Mensual,Trimestral
5,6280,5,ESE-BP-01-5,ESE-BP-01,1.2 Bienes para Transformación - Mensual,Trimestral
6,6281,6,ESE-BP-01-6,ESE-BP-01,1.3 Otros Bienes - Mensual,Trimestral
7,6282,7,ESE-BP-01-7,ESE-BP-01,2. Importaciones FOB - Mensual,Trimestral
8,6283,8,ESE-BP-01-8,ESE-BP-01,2.1 Mercancías generales - Mensual,Trimestral
9,6284,9,ESE-BP-01-9,ESE-BP-01,2.2 Bienes para Transformación - Mensual,Trimestral


### IES = Ingresos y Egresos de Servicios

Ingresos y Egresos:

* Exportaciones (Ingresos)
* Importaciones (Egresos)

Periodicidad:

* Anual
* Trimestral

Variables:

* Transporte
* Viajes
* Servicios de Comunicaciones
* Otros Servicios Empresariales
* Servicios de Gobierno
* Otros Servicios
* Servicios de Seguro
* Servicios de InformÃ¡tica e Información
* Servicios de RegalÃ­as y Derechos de Licencia

**OJO: La columna Periodicidad menciona que todo es Trimestral, sin embargo la Descripción menciona valores anuales y trimestrales.**


In [66]:
filtro = "ESE-IES-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,376,1,ESE-IES-01-1,ESE-IES-01,ESE-IES-01-1-Anual-Exportaciones (Ingresos)-Transporte,Trimestral
2,377,2,ESE-IES-01-2,ESE-IES-01,ESE-IES-01-2-Anual-Exportaciones (Ingresos)-Viajes,Trimestral
3,378,3,ESE-IES-01-3,ESE-IES-01,ESE-IES-01-3-Anual-Exportaciones (Ingresos)-Servicios de Comunicaciones,Trimestral
4,379,4,ESE-IES-01-4,ESE-IES-01,ESE-IES-01-4-Anual-Exportaciones (Ingresos)-Otros Servicios Empresariales,Trimestral
5,380,5,ESE-IES-01-5,ESE-IES-01,ESE-IES-01-5-Anual-Exportaciones (Ingresos)-Servicios de Gobierno,Trimestral
6,381,6,ESE-IES-01-6,ESE-IES-01,ESE-IES-01-6-Anual-Exportaciones (Ingresos)-Otros Servicios,Trimestral
7,382,7,ESE-IES-01-7,ESE-IES-01,ESE-IES-01-7-Anual-Importaciones (Egresos)-Transporte,Trimestral
8,383,8,ESE-IES-01-8,ESE-IES-01,ESE-IES-01-8-Anual-Importaciones (Egresos)-Viajes,Trimestral
9,384,9,ESE-IES-01-9,ESE-IES-01,ESE-IES-01-9-Anual-Importaciones (Egresos)-Servicios de Comunicaciones,Trimestral


### PII = Posición de Inversión Internacional

Agrupación:

* Posición de Inversión Internacional por Categorí­a
* Posición de los Activos Financieros con no Residentes por Componente
* Posición de los Pasivos Financieros con no Residentes por Componente
* Posición de Inversión Internacional Neta por Componente

Cuentas:

* Activos
* Pasivos
* PII Neta
* Inversió³n extranjera directa
* Inversión de cartera
* Derivados financieros
* Otra inversión
* Activos de reservas lí­quidos oficiales


In [67]:
filtro = "ESE-PII-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,480,A1,ESE-PII-01-A1,ESE-PII-01,ESE-PII-01 - A - Posición de Inversión Internacional por Categoría - Activos,Trimestral
2,481,A2,ESE-PII-01-A2,ESE-PII-01,ESE-PII-01 - A - Posición de Inversión Internacional por Categoría - Pasivos,Trimestral
3,482,A3,ESE-PII-01-A3,ESE-PII-01,ESE-PII-01 - A - Posición de Inversión Internacional por Categoría - PII Neta,Trimestral
4,483,A4,ESE-PII-01-A4,ESE-PII-01,ESE-PII-01 - B - Posición de los Activos Financieros con no Residentes por Componente - Inversión extranjera directa,Trimestral
5,484,A5,ESE-PII-01-A5,ESE-PII-01,ESE-PII-01 - B - Posición de los Activos Financieros con no Residentes por Componente - Inversión de cartera,Trimestral
6,485,A6,ESE-PII-01-A6,ESE-PII-01,ESE-PII-01 - B - Posición de los Activos Financieros con no Residentes por Componente - Derivados financieros,Trimestral
7,486,A7,ESE-PII-01-A7,ESE-PII-01,ESE-PII-01 - B - Posición de los Activos Financieros con no Residentes por Componente - Otra inversión,Trimestral
8,487,A8,ESE-PII-01-A8,ESE-PII-01,ESE-PII-01 - B - Posición de los Activos Financieros con no Residentes por Componente - Activos de reservas líquidos oficiales,Trimestral
9,488,A9,ESE-PII-01-A9,ESE-PII-01,ESE-PII-01 - C - Posición de los Pasivos Financieros con no Residentes por Componente - Inversión extranjera directa,Trimestral


### IED = Inversión Extranjera Directa


In [68]:
filtro = "ESE-IED-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,404,1,ESE-IED-01-1,ESE-IED-01,"ESE-IED-01 - Agricultura, Silvicultura, Caza y Pesca",Trimestral
2,405,10,ESE-IED-01-10,ESE-IED-01,"ESE-IED-01 - Transporte, Almacenaje y Telecomunicaciones",Trimestral
3,406,2,ESE-IED-01-2,ESE-IED-01,"ESE-IED-01 - Comercio, Restaurantes y Hoteles",Trimestral
4,407,3,ESE-IED-01-3,ESE-IED-01,ESE-IED-01 - Construcción,Trimestral
5,408,4,ESE-IED-01-4,ESE-IED-01,"ESE-IED-01 - Electricidad, Gas y Agua",Trimestral
6,409,5,ESE-IED-01-5,ESE-IED-01,ESE-IED-01 - Industria de Bienes para Transformación (Maquila),Trimestral
7,410,6,ESE-IED-01-6,ESE-IED-01,ESE-IED-01 - Industria Manufacturera,Trimestral
8,411,7,ESE-IED-01-7,ESE-IED-01,ESE-IED-01 - Minas y Canteras,Trimestral
9,412,8,ESE-IED-01-8,ESE-IED-01,ESE-IED-01 - Servicios,Trimestral


## EMF = Estadísiticas Monetarias y Financieras (Sector Fiscal)

### AMCC = Agregados Monetarios, Crédito y Captación

#### 01 Agregados Monetarios, Crédito y Captación


In [69]:
filtro = "EMF-AMCC-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,534,1,EMF-AMCC-01-1,EMF-AMCC-01,EMF-AMCC-01 - VariaciónM1 - Agregados Monetarios - M1,Mensual
2,535,2,EMF-AMCC-01-2,EMF-AMCC-01,EMF-AMCC-01 - M1 - Agregados Monetarios - M1,Mensual
3,536,3,EMF-AMCC-01-3,EMF-AMCC-01,EMF-AMCC-01 - VariaciónM2 - Agregados Monetarios - M2,Mensual
4,537,4,EMF-AMCC-01-4,EMF-AMCC-01,EMF-AMCC-01 - M2 - Agregados Monetarios - M2,Mensual
5,538,5,EMF-AMCC-01-5,EMF-AMCC-01,EMF-AMCC-01 - M3 - Agregados Monetarios - M3,Mensual
6,539,6,EMF-AMCC-01-6,EMF-AMCC-01,EMF-AMCC-01 - VariaciónM3 - Agregados Monetarios - M3,Mensual
7,540,45,EMF-AMCC-01-45,EMF-AMCC-01,EMF-AMCC-01 - VariaciónCréditodelSFTotal - Crédito Sector Financiero - Total,Mensual
8,541,27,EMF-AMCC-01-27,EMF-AMCC-01,EMF-AMCC-01 - CréditodelosBancosComercialesTotal - Crédito de los Bancos Comerciales - Total,Mensual
9,542,28,EMF-AMCC-01-28,EMF-AMCC-01,EMF-AMCC-01 - VariaciónCréditodelosBancosComercialesTotal - Crédito de los Bancos Comerciales - Total,Mensual


#### 02 Captación: Bancos Comerciales y Otras Sociedades de Depósito (Variación y Saldo)


In [70]:
filtro = "EMF-AMCC-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,621,1,EMF-AMCC-02-1,EMF-AMCC-02,Captación Bancos Comerciales-CaptaciónBCHogares-Hogares-Saldo,Mensual
2,622,2,EMF-AMCC-02-2,EMF-AMCC-02,Captación Bancos Comerciales-CaptaciónBCOtrasSociedadesnoFinancieras-Empresas Privadas-Saldo,Mensual
3,623,3,EMF-AMCC-02-3,EMF-AMCC-02,Captación Bancos Comerciales-CaptaciónBCOtrosSectores-Otros Sectores-Saldo,Mensual
4,624,4,EMF-AMCC-02-4,EMF-AMCC-02,Captación Bancos Comerciales-CaptacióndelosBancosComercialesTotal-Total-Saldo,Mensual
5,625,5,EMF-AMCC-02-5,EMF-AMCC-02,Captación Bancos Comerciales-VariaciónCaptaciónBCHogares-Hogares-Variación,Mensual
6,626,6,EMF-AMCC-02-6,EMF-AMCC-02,Captación Bancos Comerciales-VariaciónCaptaciónBCOtrasSociedadesnoFinancieras-Empresas Privadas-Variación,Mensual
7,627,7,EMF-AMCC-02-7,EMF-AMCC-02,Captación Bancos Comerciales-VariaciónCaptaciónBCOtrosSectores-Otros Sectores-Variación,Mensual
8,628,8,EMF-AMCC-02-8,EMF-AMCC-02,Captación Bancos Comerciales-VariaciónCaptaciónBCTotal-Total-Variación,Mensual
9,629,9,EMF-AMCC-02-9,EMF-AMCC-02,Captación Otras Sociedades de Depósito-CaptacióndelasOSDTotal-Total-Saldo,Mensual


#### 03 Crédito de los Bancos Comerciales


In [71]:
filtro = "EMF-AMCC-03"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,637,1,EMF-AMCC-03-1,EMF-AMCC-03,Crédito de los Bancos Comerciales-CréditoBCEmpresasPrivadas-Empresas Privadas-Saldo,Mensual
2,638,2,EMF-AMCC-03-2,EMF-AMCC-03,Crédito de los Bancos Comerciales-CréditoBCHogares-Hogares-Saldo,Mensual
3,639,3,EMF-AMCC-03-3,EMF-AMCC-03,Crédito de los Bancos Comerciales-CréditodelosBancosComercialesTotal-Total-Saldo,Mensual
4,640,4,EMF-AMCC-03-4,EMF-AMCC-03,Crédito de los Bancos Comerciales-VariaciónCréditoBCEmpresasPrivadas-Empresas Privadas-Variación,Mensual
5,641,5,EMF-AMCC-03-5,EMF-AMCC-03,Crédito de los Bancos Comerciales-VariaciónCréditoBCHogares-Hogares-Variación,Mensual
6,642,6,EMF-AMCC-03-6,EMF-AMCC-03,Crédito de los Bancos Comerciales-VariaciónCréditodelosBancosComercialesTotal-Total-Variación,Mensual


#### 04 Crédito Otras Sociedades de Depósito


In [72]:
filtro = "EMF-AMCC-04"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,643,1,EMF-AMCC-04-1,EMF-AMCC-04,Crédito Otras Sociedades de Depósito-CréditodelasOSDaEmpresasPrivadas-Empresas Privadas-Saldo,Mensual
2,644,2,EMF-AMCC-04-2,EMF-AMCC-04,Crédito Otras Sociedades de Depósito-CréditodelasOSDaHogares-Hogares-Saldo,Mensual
3,645,3,EMF-AMCC-04-3,EMF-AMCC-04,Crédito Otras Sociedades de Depósito-CréditodelasOSDTotal-Total-Saldo,Mensual
4,646,4,EMF-AMCC-04-4,EMF-AMCC-04,Crédito Otras Sociedades de Depósito-VariaciónCréditodelasOSDaEmpresasPrivadas-Empresas Privadas-Variación,Mensual
5,647,5,EMF-AMCC-04-5,EMF-AMCC-04,Crédito Otras Sociedades de Depósito-VariaciónCréditodelasOSDaHogares-Hogares-Variación,Mensual
6,648,6,EMF-AMCC-04-6,EMF-AMCC-04,Crédito Otras Sociedades de Depósito-VariaciónCréditodelasOSDTotal-Total-Variación,Mensual


#### 05 Saldo de Préstamos (Millones de Lempiras)


In [73]:
filtro = "EMF-AMCC-05"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,649,1,EMF-AMCC-05-1,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-Agropecuaria-Saldo,Mensual
2,650,2,EMF-AMCC-05-2,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-Comercio-Saldo,Mensual
3,651,3,EMF-AMCC-05-3,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-Consumo-Saldo,Mensual
4,652,4,EMF-AMCC-05-4,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-Industria-Saldo,Mensual
5,653,5,EMF-AMCC-05-5,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-PropiedadRaíz-Saldo,Mensual
6,654,6,EMF-AMCC-05-6,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-Servicios-Saldo,Mensual
7,655,7,EMF-AMCC-05-7,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-Total-Saldo,Mensual
8,656,8,EMF-AMCC-05-8,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-VariaciónAgropecuaria-Variación,Mensual
9,657,9,EMF-AMCC-05-9,EMF-AMCC-05,Saldo de préstamos-Saldos en millones de lempiras-VariaciónComercio-Variación,Mensual


De los siguientes sectores:

* Agropecuaria
* Comercio
* Consumo
* Industria
* Propiedad Raí­z
* Servicios
* Total

#### 06 Préstamos Nuevos (Flujos acumulados en millones de lempiras)


In [74]:
filtro = "EMF-AMCC-06"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,663,1,EMF-AMCC-06-1,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-Agropecuaria-Saldo,Mensual
2,664,2,EMF-AMCC-06-2,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-Comercio-Saldo,Mensual
3,665,3,EMF-AMCC-06-3,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-Consumo-Saldo,Mensual
4,666,4,EMF-AMCC-06-4,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-Industria-Saldo,Mensual
5,667,5,EMF-AMCC-06-5,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-PropiedadRaíz-Saldo,Mensual
6,668,6,EMF-AMCC-06-6,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-Servicios-Saldo,Mensual
7,669,7,EMF-AMCC-06-7,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-Total-Saldo,Mensual
8,670,8,EMF-AMCC-06-8,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-VariaciónAgropecuaria-Variación,Mensual
9,671,9,EMF-AMCC-06-9,EMF-AMCC-06,Préstamos nuevos-Flujos acumulados en millones de lempiras-VariaciónComercio-Variación,Mensual


De los siguientes sectores:

* Agropecuaria
* Comercio
* Consumo
* Industria
* Propiedad Raí­z
* Servicios
* Total

### TI = Tasas de Interés

#### 01 Tasas de Interés, Activa y Pasiva, por Moneda

Por Tipo:

* Activa Préstamos
* Pasiva Ahorros
* Pasiva Cheques
* Pasiva Plazo

Por Moneda:

* Moneda Nacional
* Moneda Extranjera


In [75]:
filtro = "EMF-TI-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,580,2,EMF-TI-01-2,EMF-TI-01,EMF-TI-01 - ActivaMonedaNacional - Activa Préstamos - Moneda Nacional,Mensual
2,581,1,EMF-TI-01-1,EMF-TI-01,EMF-TI-01 - ActivaMonedaExtranjera - Activa Préstamos - Moneda Extranjera,Mensual
3,582,6,EMF-TI-01-6,EMF-TI-01,EMF-TI-01 - PasivadeAhorroMonedaNacional - Pasiva Ahorros - Moneda Nacional,Mensual
4,583,5,EMF-TI-01-5,EMF-TI-01,EMF-TI-01 - PasivadeAhorroMonedaExtranjera - Pasiva Ahorros - Moneda Extranjera,Mensual
5,584,8,EMF-TI-01-8,EMF-TI-01,EMF-TI-01 - PasivadeChequesMonedaNacional - Pasiva Cheques - Moneda Nacional,Mensual
6,585,7,EMF-TI-01-7,EMF-TI-01,EMF-TI-01 - PasivadeChequesMonedaExtranjera - Pasiva Cheques - Moneda Extranjera,Mensual
7,586,4,EMF-TI-01-4,EMF-TI-01,EMF-TI-01 - PasivaaPlazoMonedaNacional - Pasiva Plazo - Moneda Nacional,Mensual
8,587,3,EMF-TI-01-3,EMF-TI-01,EMF-TI-01 - PasivaaPlazoMonedaExtranjera - Pasiva Plazo - Moneda Extranjera,Mensual


#### 02 Por Actividad

Por Moneda:

* Moneda Nacional
* Moneda Extranjera


In [76]:
filtro = "EMF-TI-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,588,1,EMF-TI-02-1,EMF-TI-02,"EMF-TI-02 - Nacional - Agricultura, Ganadería, Silvicultura y Pesca",Mensual
2,589,12,EMF-TI-02-12,EMF-TI-02,EMF-TI-02 - Nacional - Explotación de Minas y Canteras,Mensual
3,590,14,EMF-TI-02-14,EMF-TI-02,EMF-TI-02 - Nacional - Industrias Manufactureras,Mensual
4,591,15,EMF-TI-02-15,EMF-TI-02,EMF-TI-02 - Nacional - Construcción,Mensual
5,592,16,EMF-TI-02-16,EMF-TI-02,EMF-TI-02 - Nacional - Consumo,Mensual
6,593,17,EMF-TI-02-17,EMF-TI-02,EMF-TI-02 - Nacional - Comercio,Mensual
7,594,18,EMF-TI-02-18,EMF-TI-02,"EMF-TI-02 - Nacional - Transporte, Almacenamiento, Información y Comunicaciones",Mensual
8,595,19,EMF-TI-02-19,EMF-TI-02,EMF-TI-02 - Nacional - Actividades Alojamiento y Servicios Comida,Mensual
9,596,20,EMF-TI-02-20,EMF-TI-02,EMF-TI-02 - Nacional - Vivienda,Mensual


#### 03 Tasas de Interés Anuales

* Tasas de Interés Anuales Mí­nimas y Máximas del Sistema Bancario Comercial
* Tasas de Interés Anuales Promedio Ponderadas del Sistema Financiero

Por tipo:
Activa
Pasiva

Por Unidad de Medida:

* Máxima
* Mínima
* Promedio
* Por Moneda:

* Nacional
* Extranjera


In [77]:
filtro = "EMF-TI-03"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,689,1,EMF-TI-03-1,EMF-TI-03,EMF-TI-03-1-Activa Mínima en Moneda Nacional - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C1,Mensual
2,690,2,EMF-TI-03-2,EMF-TI-03,EMF-TI-03-2-Activa Máxima en Moneda Nacional - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C2,Mensual
3,691,3,EMF-TI-03-3,EMF-TI-03,EMF-TI-03-3-Pasiva Mínima Plazo 180 días Moneda Nacional - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C3,Mensual
4,692,4,EMF-TI-03-4,EMF-TI-03,EMF-TI-03-4-Pasiva Máxima Plazo 180 días Moneda Nacional - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C4,Mensual
5,693,5,EMF-TI-03-5,EMF-TI-03,EMF-TI-03-5-Activa Mínima en Moneda Extranjera - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C5,Mensual
6,694,6,EMF-TI-03-6,EMF-TI-03,EMF-TI-03-6-Activa Máxima en Moneda Extranjera - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C6,Mensual
7,695,7,EMF-TI-03-7,EMF-TI-03,EMF-TI-03-7-Pasiva Mínima Plazo 180 días Moneda Extranjera - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C7,Mensual
8,696,8,EMF-TI-03-8,EMF-TI-03,EMF-TI-03-8-Pasiva Máxima Plazo 180 días Moneda Extranjera - Tasas de Interés Anuales Mínimas y Máximas del Sistema Bancario Comercial-C8,Mensual
9,697,9,EMF-TI-03-9,EMF-TI-03,EMF-TI-03-9-Activa Promedio Ponderada en Moneda Nacional - Tasas de Interés Anuales Promedio Ponderadas del Sistema Financiero-C9,Mensual


### PFS = Panorama de las Sociedades Financieras

#### 01 Bancos Comerciales, Activos y Pasivos Externos


In [78]:
filtro = "EMF-PFS-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,677,1,EMF-PFS-01-1,EMF-PFS-01,Activos Externos Netos-Bancos Comerciales,Mensual
2,678,2,EMF-PFS-01-2,EMF-PFS-01,Activos Externos-Bancos Comerciales,Mensual
3,679,3,EMF-PFS-01-3,EMF-PFS-01,Pasivos Externos-Bancos Comerciales,Mensual


#### 02 Activos y Pasivos Externos


In [79]:
filtro = "EMF-PFS-02"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,680,1,EMF-PFS-02-1,EMF-PFS-02,Activos Externos Netos-Banco Central de Honduras,Mensual
2,681,2,EMF-PFS-02-2,EMF-PFS-02,Activos Externos Netos-Otras Sociedades de Depósito,Mensual
3,682,3,EMF-PFS-02-3,EMF-PFS-02,Activos Externos Netos-Sociedades de Depósito,Mensual
4,683,4,EMF-PFS-02-4,EMF-PFS-02,Activos Externos-Banco Central de Honduras,Mensual
5,684,5,EMF-PFS-02-5,EMF-PFS-02,Activos Externos-Otras Sociedades de Depósito,Mensual
6,685,6,EMF-PFS-02-6,EMF-PFS-02,Activos Externos-Sociedades de Depósito,Mensual
7,686,7,EMF-PFS-02-7,EMF-PFS-02,Pasivos Externos-Banco Central de Honduras,Mensual
8,687,8,EMF-PFS-02-8,EMF-PFS-02,Pasivos Externos-Otras Sociedades de Depósito,Mensual
9,688,9,EMF-PFS-02-9,EMF-PFS-02,Pasivos Externos-Sociedades de Depósito,Mensual


### EBM = Emisión y Base Monetaria

* Saldo
* Variación


In [80]:
filtro = "EMF-EBM-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,518,1,EMF-EBM-01-1,EMF-EBM-01,EMF-EBM-01-Emision Monetaria-Saldo,Mensual
2,519,4,EMF-EBM-01-4,EMF-EBM-01,EMF-EBM-01-Emision Monetaria-Variación,Mensual
3,520,2,EMF-EBM-01-2,EMF-EBM-01,EMF-EBM-01-Base Monetaria Restringida-Saldo,Mensual
4,521,5,EMF-EBM-01-5,EMF-EBM-01,EMF-EBM-01-Base Monetaria Restringida-Variación,Mensual
5,522,3,EMF-EBM-01-3,EMF-EBM-01,EMF-EBM-01-Base Monetaria Amplia-Saldo,Mensual
6,523,6,EMF-EBM-01-6,EMF-EBM-01,EMF-EBM-01-Base Monetaria Amplia-Variación,Mensual


## EM = Estadísticas Monetarias (TPM y RIN)

### Reservas Internacionales Netas (RIN)

* RIN
* Activos de Reserva Oficial


In [81]:
filtro = "EM-RIN-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,524,1,EM-RIN-01-1,EM-RIN-01,EM-RIN-01 - Reservas Internacionales Netas - Saldo,Mensual
2,525,3,EM-RIN-01-3,EM-RIN-01,EM-RIN-01 - Reservas Internacionales Netas - Variación Absoluta Mensual,Mensual
3,526,4,EM-RIN-01-4,EM-RIN-01,EM-RIN-01 - Reservas Internacionales Netas - Variación Absoluta Acumulada,Mensual
4,527,5,EM-RIN-01-5,EM-RIN-01,EM-RIN-01 - Reservas Internacionales Netas - Variación Absoluta Interanual,Mensual
5,528,6,EM-RIN-01-6,EM-RIN-01,EM-RIN-01 - Reservas Internacionales Netas - Variación Interanual en porcentajes,Mensual
6,529,7,EM-RIN-01-7,EM-RIN-01,EM-RIN-01 - Activos de Reserva Oficial - Saldo,Mensual
7,530,8,EM-RIN-01-8,EM-RIN-01,EM-RIN-01 - Activos de Reserva Oficial - Variación Absoluta Mensual,Mensual
8,531,9,EM-RIN-01-9,EM-RIN-01,EM-RIN-01 - Activos de Reserva Oficial - Variación Absoluta Acumulada,Mensual
9,532,10,EM-RIN-01-10,EM-RIN-01,EM-RIN-01 - Activos de Reserva Oficial - Variación Absoluta Interanual,Mensual


### TPM


In [82]:
filtro = "EM-TPM-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,699,1,EM-TPM-01-1,EM-TPM-01,Tasa Politica Monetaria - Diario,Diario
2,700,2,EM-TPM-01-2,EM-TPM-01,Tasa Politica Monetaria - Mensual,Mensual


## ESP = Estadísticas de Sistema de Pagos

### ACH = Operaciones de Crédito Compensadas en Ceproban y Liquidadas en BCH


In [83]:
filtro = "ESP-ACH-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,611,1,ESP-ACH-01-1,ESP-ACH-01,ESP-ACH-01-1-Cantidad-Dólares,Mensual
2,612,2,ESP-ACH-01-2,ESP-ACH-01,ESP-ACH-01-2-Cantidad-Lempiras,Mensual
3,613,3,ESP-ACH-01-3,ESP-ACH-01,ESP-ACH-01-3-Monto-Dólares,Mensual
4,614,4,ESP-ACH-01-4,ESP-ACH-01,ESP-ACH-01-4-Monto-Lempiras,Mensual


### CCECH = Cheques Compensados en Ceproban y Liquidados en BCH


In [84]:
filtro = "ESP-CCECH-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,615,1,ESP-CCECH-01-1,ESP-CCECH-01,ESP-CCECH-01-1-Cantidad-Dólares,Mensual
2,616,2,ESP-CCECH-01-2,ESP-CCECH-01,ESP-CCECH-01-2-Cantidad-Lempiras,Mensual
3,617,3,ESP-CCECH-01-3,ESP-CCECH-01,ESP-CCECH-01-3-Monto-Dólares,Mensual
4,618,4,ESP-CCECH-01-4,ESP-CCECH-01,ESP-CCECH-01-4-Monto-Lempiras,Mensual


## EC = Estadísticas de Tipo de Cambio

### TCR = Tipo de Cambio de Referencia


In [85]:
filtro = "EC-TCR"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,97,01,EC-TCR-01,EC-TCR,Estadísticas Cambiarias - Tipo de Cambio de Referencia,Diario


### TCN = Tipo de Cambio Nominal


In [86]:
filtro = "EC-TCN-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,619,1,EC-TCN-01-1,EC-TCN-01,Tipo de Cambio Nominal - Compra,Diario
2,620,2,EC-TCN-01-2,EC-TCN-01,Tipo de Cambio Nominal - Venta,Diario


## EP = Estadísticas de Precios

### IPC Índice de Precios al Consumidor


In [87]:
filtro = "EP-IPC-01"
url = "https://bchapi-am.azure-api.net/api/v1/indicadores/grupo/" * filtro * "?formato=Json"
get_groups_filter(key,url)

Row,Id,CorrelativoGrupo,Nombre,Grupo,Descripcion,Periodicidad
,Int64,String,String,String,String,String
1,608,1,EP-IPC-01-1,EP-IPC-01,EP-IPC-01 - Índice,Mensual
2,609,2,EP-IPC-01-2,EP-IPC-01,EP-IPC-01 - Variación interanual,Mensual
3,610,3,EP-IPC-01-3,EP-IPC-01,EP-IPC-01 - Variación mensual,Mensual


# Descargar Indicadores por Rangos


In [88]:
# ids = 11001:11519 # Total de indicadores: 11519
# get_data_indicators(key,ids)

In [89]:
ids = 11519 # Total de indicadores: 11519
get_data_indicator(key,ids)

Row,IndicadorId,Nombre,Descripcion,Fecha,Valor
,Int64,String,String,Date,Float64
1,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2023-01-01,11.7
2,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2022-01-01,18.8
3,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2021-01-01,10.4
4,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2020-01-01,-0.4
5,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2019-01-01,5.4
6,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2018-01-01,2.1
7,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2017-01-01,4.2
8,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2016-01-01,6.6
9,11519,ESR-ODA-01-94,OFERTA GLOBAL-VALOR AGREGADO BRUTO-MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)-VARIACIONES RELATIVAS CORRIENTES,2015-01-01,10.1


**OJO: las columnas 6642 hasta 6687 tienen error: column name "Fecha" not found in the data frame since it has no columns**
**OJO: las columnas 7392 hasta 7446 tienen error: column name "Fecha" not found in the data frame since it has no columns**


In [90]:
key = clave
df = get_indicators(key)

Row,Id,Nombre,Descripcion,CorrelativoGrupo,Grupo,Periodicidad
,Int64,String,String,String,String,String
1,1,ESR-PIBT-PROD-DES-CONST-15,Serie Desestacionalizada-VALORES CONSTANTES-21-Valor Agregado Bruto a precios básicos,15,ESR-PIBT-PROD-DES-CONST,Trimestral
2,2,ESR-PIBT-PROD-DES-CORR-12,Serie Desestacionalizada-VALORES CORRIENTES-14-Servicios de Enseñanza Servicios Sociales y de Salud,12,ESR-PIBT-PROD-DES-CORR,Trimestral
3,3,ESR-PIBT-PROD-DES-CORR-13,Serie Desestacionalizada-VALORES CORRIENTES-16-Servicios Comunitarios Sociales y Personales,13,ESR-PIBT-PROD-DES-CORR,Trimestral
4,4,ESR-PIBT-PROD-DES-CORR-14,Serie Desestacionalizada-VALORES CORRIENTES-20-Menos Servicios de Intermediación Financiera Medidos Indirectamente (SIFMI),14,ESR-PIBT-PROD-DES-CORR,Trimestral
5,5,ESR-PIBT-PROD-DES-CORR-15,Serie Desestacionalizada-VALORES CORRIENTES-21-Valor Agregado Bruto a precios básicos,15,ESR-PIBT-PROD-DES-CORR,Trimestral
6,6,ESR-PIBT-PROD-DES-CORR-16,Serie Desestacionalizada-VALORES CORRIENTES-22-Más Impuestos netos de subvenciones sobre la producción y las importaciones,16,ESR-PIBT-PROD-DES-CORR,Trimestral
7,7,ESR-PIBT-PROD-DES-CORR-17,Serie Desestacionalizada-VALORES CORRIENTES-23-Producto Interno Bruto a precios de mercado,17,ESR-PIBT-PROD-DES-CORR,Trimestral
8,8,ESR-PIBT-GAST-OG-CONST-1,Serie Original-VALORES CONSTANTES-1-Gastos de Consumo Final,1,ESR-PIBT-GAST-OG-CONST,Trimestral
9,9,ESR-PIBT-GAST-OG-CONST-2,Serie Original-VALORES CONSTANTES-2-Sector Privado,2,ESR-PIBT-GAST-OG-CONST,Trimestral
